In [1]:
import geopandas as gp
from shapely.geometry import LineString, Point
import rtree
import os
import numpy as np

In [2]:
from itertools import chain

In [126]:
dc_segments = gp.read_file("../dc-maps-master/maps/street-segments.geojson")
vision0_pts = gp.read_file("../dc-maps-master/maps/vision-zero-safety-transportation.geojson")

In [4]:
print(vision0_pts.info())
vision0_pts.head().T

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5026 entries, 0 to 5025
Data columns (total 10 columns):
COMMENTS       4536 non-null object
GLOBALID       5026 non-null object
OBJECTID       5026 non-null int64
REQUESTDATE    5019 non-null object
REQUESTID      5015 non-null object
REQUESTTYPE    5026 non-null object
STATUS         5019 non-null object
STREETSEGID    5017 non-null float64
USERTYPE       5020 non-null object
geometry       5026 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 392.7+ KB
None


,0,1,2,3,4
COMMENTS,Intersection dangerous for Peds at rush hour +...,WMATA vehicles regularly use the sidewalk to p...,None,Vehicles - including WMATA buses - regularly g...,None
GLOBALID,{148BFB13-CC65-46F7-B347-7A69C77081EE},{F2A0AFC7-2381-4F08-AEB6-55E9E545ED64},{4926AE04-E684-4B5E-9A26-12351447E645},{E2AA1B8E-7748-4530-9D71-B880F5EBE398},{FA779B8C-7B3D-4559-BD45-EE8FC85CB012}
OBJECTID,48,49,50,51,52
REQUESTDATE,2015-06-29T20:33:51,2015-06-29T20:34:10,2015-06-29T20:34:31,2015-06-29T20:34:33,2015-06-29T20:34:48
REQUESTID,48,49,50,51,52
REQUESTTYPE,Failure to stop for pedestrians,Other Driving Issue,Failure to stop for pedestrians,Speeding,Failure to stop for pedestrians
STATUS,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned
STREETSEGID,11875,2016,13281,9827,9162
USERTYPE,Pedestrian,Pedestrian,Pedestrian,Biker,Pedestrian
geometry,POINT (-77.02962759391977 38.9048501530145),POINT (-77.00349975118203 38.90565930795784),POINT (-77.02190330185668 38.91868690690595),POINT (-77.08572889213174 38.90869297027604),POINT (-77.02294848571017 38.9166450535154)


In [5]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    from math import radians, cos, sin, asin, sqrt
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    m = km *1000
    return m   
def ptfromln(pt, ln):
    n_pt = ln.interpolate(ln.project(pt))
    lon1, lat1 = n_pt.coords[0]
    lon2, lat2 = pt.coords[0]
    return haversine(lon1, lat1, lon2, lat2)
def pts2seg(pts, gp_segs, buffer_dis=50, near_dis_thres=5):
    pts_crs,gp_segs_crs = pts.to_crs(epsg=3559), gp_segs.to_crs(epsg=3559)
    pts_crs_bfr = pts_crs.copy()

    pts_crs_bfr.geometry = pts_crs_bfr.buffer(near_dis_thres*1.1)
    close_jn = gp.tools.sjoin(pts_crs_bfr, gp_segs_crs)[['OBJECTID_left','STREETSEGID_right']]
    handledid = set(pd.unique(close_jn.OBJECTID_left))
    mask = (~pts_crs_bfr.OBJECTID.isin(handledid))

    far_jns = []
    while pts_crs_bfr[mask].shape[0]!=0:
        pts_crs_bfr.loc[mask, 'geometry'] = pts_crs_bfr[mask].buffer(buffer_dis)
        jn = gp.tools.sjoin(pts_crs_bfr[mask], gp_segs_crs)[['OBJECTID_left','STREETSEGID_right']]
        far_jns.append(jn)
        handledid |= set(pd.unique(jn.OBJECTID_left))
        mask = (~pts_crs_bfr.OBJECTID.isin(handledid))
        
    far_jns = pd.concat(far_jns)
    mr = pd.merge(gp_segs[['geometry','STREETSEGID']],far_jns , left_on='STREETSEGID', right_on='STREETSEGID_right')
    mr = pd.merge(pts[['OBJECTID','geometry','STREETSEGID']],mr, left_on='OBJECTID', right_on='OBJECTID_left')
    mr['dis']=mr.apply(lambda x: ptfromln(x.geometry_x, x.geometry_y),axis=1)
    result = close_jn.groupby('OBJECTID_left')['STREETSEGID_right'].apply(list).append(mr.groupby('OBJECTID').apply(lambda x: [x.ix[x.dis.idxmin()].STREETSEGID_y]))
    return pd.DataFrame(result, columns=['segid'])

In [6]:
dc_segs_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]

In [7]:
dc_segs_statistic = dc_segs_statistic.merge(vision0_pts, left_on = 'STREETSEGID', right_index=True, how='left')

In [8]:
dc_segs_statistic.shape

(13522, 14)

In [9]:
dc_segs_statistic = dc_segs_statistic.dropna()

In [10]:
dc_segs_statistic.head().T

,0,4,16,19,25
OBJECTID_x,1,5,17,20,26
STREETSEGID_x,876,2216,2902,4213,179
SHAPE_Length,192.925,119.19,126.698,119.102,153.46
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01161670521323 38.956595348877...,LINESTRING (-77.01324339541158 38.951215745942...,LINESTRING (-77.01357027112063 38.954497383391...,LINESTRING (-77.01368810906661 38.961335901924...
COMMENTS,Cyclists constantly use the sidewalk up and do...,Traffic going up this hill is generally too fa...,This intersection has a fair number of pedestr...,Vehicles picking up and dropping off passenger...,I was hit by a car here when I was in the cycl...
GLOBALID,{17DD828B-8D39-4B7A-88EB-2125E60DA1EA},{BD16F1DE-92F0-4B6B-ACEA-BA904EF18683},{15AC14A9-746F-4F75-B2FA-C173260C5855},{0F7D5B3F-267F-4D2C-9D53-71D06B2F6D5C},{20B961FB-7304-44AE-BCC5-775E17675DA6}
OBJECTID_y,955,3299,1974,5688,122
REQUESTDATE,2015-07-02T12:58:50,2015-07-17T19:58:09,2015-07-10T02:30:00,2015-08-15T22:42:59,2015-06-29T20:53:04
REQUESTID,955,3299,1974,5688,122
REQUESTTYPE,Cyclist behavior,Speeding,Failure to stop for pedestrians,Double parking,Other Driving Issue


In [11]:
dc_segs_statistic[dc_segs_statistic.STREETSEGID_x == 2216]

,OBJECTID_x,STREETSEGID_x,SHAPE_Length,geometry_x,COMMENTS,GLOBALID,OBJECTID_y,REQUESTDATE,REQUESTID,REQUESTTYPE,STATUS,STREETSEGID_y,USERTYPE,geometry_y
4,5,2216,119.190312,LINESTRING (-77.01161670521323 38.956595348877...,Traffic going up this hill is generally too fa...,{BD16F1DE-92F0-4B6B-ACEA-BA904EF18683},3299.0,2015-07-17T19:58:09,3299,Speeding,Unassigned,5942.0,Biker,POINT (-77.03185508121238 38.9208823767754)


In [12]:
dc_segs_statistic

,OBJECTID_x,STREETSEGID_x,SHAPE_Length,geometry_x,COMMENTS,GLOBALID,OBJECTID_y,REQUESTDATE,REQUESTID,REQUESTTYPE,STATUS,STREETSEGID_y,USERTYPE,geometry_y
0,1,876,192.925131,LINESTRING (-77.01239193659748 38.950836102255...,Cyclists constantly use the sidewalk up and do...,{17DD828B-8D39-4B7A-88EB-2125E60DA1EA},955.0,2015-07-02T12:58:50,955,Cyclist behavior,Unassigned,11399.0,Pedestrian,POINT (-77.03195395364403 38.90678948383844)
4,5,2216,119.190312,LINESTRING (-77.01161670521323 38.956595348877...,Traffic going up this hill is generally too fa...,{BD16F1DE-92F0-4B6B-ACEA-BA904EF18683},3299.0,2015-07-17T19:58:09,3299,Speeding,Unassigned,5942.0,Biker,POINT (-77.03185508121238 38.9208823767754)
16,17,2902,126.698028,LINESTRING (-77.01324339541158 38.951215745942...,This intersection has a fair number of pedestr...,{15AC14A9-746F-4F75-B2FA-C173260C5855},1974.0,2015-07-10T02:30:00,1974,Failure to stop for pedestrians,Unassigned,7465.0,Pedestrian,POINT (-77.02606355519816 38.9686399653321)
19,20,4213,119.102319,LINESTRING (-77.01357027112063 38.954497383391...,Vehicles picking up and dropping off passenger...,{0F7D5B3F-267F-4D2C-9D53-71D06B2F6D5C},5688.0,2015-08-15T22:42:59,5688,Double parking,Unassigned,233.0,Car Driver,POINT (-77.00434504379416 38.88604499110777)
25,26,179,153.460480,LINESTRING (-77.01368810906661 38.961335901924...,I was hit by a car here when I was in the cycl...,{20B961FB-7304-44AE-BCC5-775E17675DA6},122.0,2015-06-29T20:53:04,122,Other Driving Issue,Unassigned,1067.0,Biker,POINT (-77.03240650725611 38.89551047123661)
35,36,277,205.857560,LINESTRING (-77.01368063507392 38.900211463306...,An unsignalized pedestrian crossing should be ...,{2A1C21F0-8AFD-4168-9E7F-4CB6CEFB709C},281.0,2015-06-30T14:43:34,281,Other Walking Issue,Unassigned,496.0,Pedestrian,POINT (-77.03024304187144 38.90858849023896)
39,40,1440,64.684222,LINESTRING (-77.01427336215276 38.914996139209...,As soon as this section of Potomac because 2-3...,{09BEF8F1-9B67-416F-8F89-EBC8557597AC},2818.0,2015-07-16T12:33:09,2818,Speeding,Unassigned,1114.0,Pedestrian,POINT (-76.97900052357966 38.88302359375337)
42,43,1750,79.779456,LINESTRING (-77.0144223920466 38.9162872196654...,Trucks/delivery vehicles routinely park in the...,{735F6A22-06DC-41AB-B062-12A10CC6CEAA},3062.0,2015-07-16T20:19:44,3062,Double parking,Unassigned,7775.0,Biker,POINT (-77.04293670047392 38.90725022523262)
45,46,667,61.926531,LINESTRING (-77.01462766975995 38.918114804591...,Red lights along this stretch of 4th Street. a...,{48682AFE-D20C-40B4-8B4C-A775DBE5E393},642.0,2015-07-01T17:10:13,642,Other Biking Issue,Unassigned,6574.0,Biker,POINT (-77.01755746826102 38.88606030948256)
47,48,2308,54.452949,LINESTRING (-77.01475138888762 38.919182095956...,There are numerous potholes on this street whi...,{088DF38A-9886-4987-8D71-EE394131BD3F},3971.0,2015-07-28T13:13:04,3971,Other Driving Issue,Unassigned,11085.0,Pedestrian,POINT (-76.9536437737025 38.88724114859725)


In [13]:
dc_segs_statistic[dc_segs_statistic.STREETSEGID_x == 644]

,OBJECTID_x,STREETSEGID_x,SHAPE_Length,geometry_x,COMMENTS,GLOBALID,OBJECTID_y,REQUESTDATE,REQUESTID,REQUESTTYPE,STATUS,STREETSEGID_y,USERTYPE,geometry_y
4435,4436,644,275.843424,LINESTRING (-77.03848871583934 38.911133414859...,South-side sidewalk from 6th to 9th is impassi...,{37A25004-7921-4DB9-B02C-B7CACEED222F},609.0,2015-07-01T16:56:46,609,Accessibility Issue,Unassigned,13176.0,Pedestrian,POINT (-76.99616576427324 38.90524480982609)


In [14]:
dc_segs_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]
dc_segs_statistic = dc_segs_statistic.merge(vision0_pts, left_on = 'STREETSEGID', right_index=True, how='left')

In [15]:
import pandas as pd

In [16]:
vision0_pts_seg = pts2seg(vision0_pts, dc_segments)

In [17]:
seg_counts = pd.DataFrame(pd.Series(list(chain.from_iterable(vision0_pts_seg.values.ravel()))).value_counts(),columns=["vision0_pts_seg"])

In [18]:
seg_counts.head().T

,14208,11616,8231,9549,746
vision0_pts_seg,24,19,18,16,16


In [19]:
dc_segs_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]
dc_segs_statistic = dc_segs_statistic.merge(seg_counts, left_on = 'STREETSEGID', right_on='STREETSEGID', right_index=True, how='left')

In [ ]:
vi

In [20]:
vision0_pts.head().T

,0,1,2,3,4
COMMENTS,Intersection dangerous for Peds at rush hour +...,WMATA vehicles regularly use the sidewalk to p...,None,Vehicles - including WMATA buses - regularly g...,None
GLOBALID,{148BFB13-CC65-46F7-B347-7A69C77081EE},{F2A0AFC7-2381-4F08-AEB6-55E9E545ED64},{4926AE04-E684-4B5E-9A26-12351447E645},{E2AA1B8E-7748-4530-9D71-B880F5EBE398},{FA779B8C-7B3D-4559-BD45-EE8FC85CB012}
OBJECTID,48,49,50,51,52
REQUESTDATE,2015-06-29T20:33:51,2015-06-29T20:34:10,2015-06-29T20:34:31,2015-06-29T20:34:33,2015-06-29T20:34:48
REQUESTID,48,49,50,51,52
REQUESTTYPE,Failure to stop for pedestrians,Other Driving Issue,Failure to stop for pedestrians,Speeding,Failure to stop for pedestrians
STATUS,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned
STREETSEGID,11875,2016,13281,9827,9162
USERTYPE,Pedestrian,Pedestrian,Pedestrian,Biker,Pedestrian
geometry,POINT (-77.02962759391977 38.9048501530145),POINT (-77.00349975118203 38.90565930795784),POINT (-77.02190330185668 38.91868690690595),POINT (-77.08572889213174 38.90869297027604),POINT (-77.02294848571017 38.9166450535154)


In [21]:
vision0_seg_gb = pd.DataFrame(vision0_pts.groupby(['STREETSEGID']).size().reset_index())
vision0_seg_gb.columns = ['STREETSEGID', 'vision0_pts_seg']
print(vision0_seg_gb.shape)
vision0_seg_gb.head().T

(2839, 2)


,0,1,2,3,4
STREETSEGID,3.0,11.0,14.0,15.0,22.0
vision0_pts_seg,2.0,1.0,5.0,2.0,1.0


In [22]:
vision0_totalcount = pd.DataFrame(vision0_seg_gb.fillna(0))
print(vision0_totalcount.shape)
vision0_totalcount.head().T

(2839, 2)


,0,1,2,3,4
STREETSEGID,3.0,11.0,14.0,15.0,22.0
vision0_pts_seg,2.0,1.0,5.0,2.0,1.0


In [23]:
dc_segs_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]
dc_segs_statistic = dc_segs_statistic.merge(vision0_totalcount, left_on = 'STREETSEGID', right_on='STREETSEGID', right_index=False, how='left')

In [75]:
def normalize(col_name, df):
    col_name_norm = col_name + '_norm'
    df[col_name_norm] = df[col_name]+1
    df[col_name_norm] = np.log2(df[col_name_norm])
    df[col_name_norm] = df[col_name_norm]/df[col_name_norm].max()

In [25]:
dc_segs_statistic['vision0_pts/len'] = dc_segs_statistic.vision0_pts_seg/dc_segs_statistic.SHAPE_Length
normalize('vision0_pts_seg', dc_segs_statistic)
normalize('vision0_pts/len', dc_segs_statistic)

In [26]:
vision0_typemerge = pd.DataFrame(vision0_pts.groupby(['STREETSEGID', 'REQUESTTYPE']).size().reset_index())

In [27]:
vision0_type_counts = vision0_pts.groupby(['STREETSEGID', 'REQUESTTYPE']).count()

In [28]:
vision0_typemerge.columns = ['STREETSEGID', 'REQUESTTYPE', 'count']

In [29]:
vision0_typemerge[vision0_typemerge.STREETSEGID==10728]

,STREETSEGID,REQUESTTYPE,count
3482,10728.0,Failure to stop for pedestrians,5
3483,10728.0,Poor visibility,3
3484,10728.0,Speeding,1


In [30]:
unique_types = set()
for req_type in vision0_typemerge.REQUESTTYPE:
    unique_types.add(req_type)
print(unique_types)

{'Other Driving Issue', 'Blocking the bikebox', 'Other Biking Issue', 'Accessibility Issue', 'Poor visibility', 'Failure to stop for pedestrians', 'Cyclist behavior', 'Long wait to cross', 'Long distance to cross', 'Speeding', 'Double parking', 'Other Walking Issue', 'Stop sign running', 'Red light running', 'Blocking the crosswalk', 'Jaywalking', 'Not enough time to cross'}


In [31]:
normalize('vision0_pts_seg', dc_segs_statistic_drop)
normalize('vision0_pts/len', dc_segs_statistic_drop)

NameError: name 'dc_segs_statistic_drop' is not defined

In [32]:
vision0_seg_stats = vision0_typemerge[['STREETSEGID']]

In [33]:
for utype in unique_types:
    vision0_type_data = vision0_typemerge[vision0_typemerge.REQUESTTYPE==utype]
    utype_count = utype + '_count'
    vision0_type_count = vision0_type_data[['STREETSEGID', 'count']]
    vision0_type_count.columns = ['STREETSEGID', utype_count]
    vision0_seg_stats = vision0_seg_stats.merge(vision0_type_count, left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=False, how='left')

In [34]:
print(vision0_seg_stats.shape)
vision0_details = pd.DataFrame(vision0_seg_stats.fillna(0))
vision0_details.head().T

(4388, 18)


,0,1,2,3,4
STREETSEGID,3.0,11.0,14.0,14.0,14.0
Other Driving Issue_count,0.0,0.0,2.0,2.0,2.0
Blocking the bikebox_count,0.0,0.0,0.0,0.0,0.0
Other Biking Issue_count,2.0,0.0,0.0,0.0,0.0
Accessibility Issue_count,0.0,0.0,1.0,1.0,1.0
Poor visibility_count,0.0,0.0,1.0,1.0,1.0
Failure to stop for pedestrians_count,0.0,0.0,0.0,0.0,0.0
Cyclist behavior_count,0.0,0.0,0.0,0.0,0.0
Long wait to cross_count,0.0,0.0,0.0,0.0,0.0
Long distance to cross_count,0.0,0.0,0.0,0.0,0.0


In [35]:
vision0_poorviz = vision0_typemerge[vision0_typemerge.REQUESTTYPE=='Poor visibility']

In [36]:
vision0_poorviz_count = vision0_poorviz[['STREETSEGID', 'count']]
vision0_poorviz_count.columns = ['STREETSEGID', 'poor_visibility_count']
vision0_poorviz_count.head().T

,5,21,23,44,80
STREETSEGID,14.0,59.0,70.0,111.0,184.0
poor_visibility_count,1.0,2.0,1.0,2.0,1.0


In [37]:
dc_segs_statistic.head().T

,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
vision0_pts_seg,2,1,NaN,NaN,NaN
vision0_pts/len,0.0103667,0.00757945,NaN,NaN,NaN
vision0_pts_seg_norm,0.350379,0.221065,NaN,NaN,NaN
vision0_pts/len_norm,0.0301949,0.022107,NaN,NaN,NaN


In [38]:
dc_segs_statistic_detailed = dc_segs_statistic.merge(vision0_details, left_on = 'STREETSEGID', right_on='STREETSEGID', right_index=False, how='left')

In [39]:
print(dc_segs_statistic_detailed.shape)
print(dc_segs_statistic_detailed.dropna().shape)
dc_segs_statistic_detailed.head().T

(15071, 25)
(4388, 25)


,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
vision0_pts_seg,2,1,NaN,NaN,NaN
vision0_pts/len,0.0103667,0.00757945,NaN,NaN,NaN
vision0_pts_seg_norm,0.350379,0.221065,NaN,NaN,NaN
vision0_pts/len_norm,0.0301949,0.022107,NaN,NaN,NaN
Other Driving Issue_count,0,0,NaN,NaN,NaN
Blocking the bikebox_count,0,0,NaN,NaN,NaN


In [40]:
dc_vision0_details = pd.DataFrame(dc_segs_statistic_detailed.fillna(0))
print(dc_vision0_details.shape)
dc_vision0_details.head().T

(15071, 25)


,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
vision0_pts_seg,2,1,0,0,0
vision0_pts/len,0.0103667,0.00757945,0,0,0
vision0_pts_seg_norm,0.350379,0.221065,0,0,0
vision0_pts/len_norm,0.0301949,0.022107,0,0,0
Other Driving Issue_count,0,0,0,0,0
Blocking the bikebox_count,0,0,0,0,0


In [41]:
for utype in unique_types:
    utype_count = utype + '_count'
    utype_countlen = utype + '_count/len'
    dc_vision0_details[utype_countlen] = dc_vision0_details[utype_count]/dc_vision0_details.SHAPE_Length
    normalize(utype_count, dc_vision0_details)
    normalize(utype_countlen, dc_vision0_details)

In [42]:
dc_vision0_details[dc_vision0_details.vision0_pts_seg==10].T

,4623,4624,4625,4626,4627,13472,13473,13474,13475
OBJECTID,3966,3966,3966,3966,3966,12046,12046,12046,12046
STREETSEGID,12238,12238,12238,12238,12238,13955,13955,13955,13955
SHAPE_Length,269.67,269.67,269.67,269.67,269.67,205.873,205.873,205.873,205.873
geometry,LINESTRING (-77.00905099283175 38.913351308004...,LINESTRING (-77.00905099283175 38.913351308004...,LINESTRING (-77.00905099283175 38.913351308004...,LINESTRING (-77.00905099283175 38.913351308004...,LINESTRING (-77.00905099283175 38.913351308004...,LINESTRING (-77.00588929348585 38.907243316708...,LINESTRING (-77.00588929348585 38.907243316708...,LINESTRING (-77.00588929348585 38.907243316708...,LINESTRING (-77.00588929348585 38.907243316708...
vision0_pts_seg,10,10,10,10,10,10,10,10,10
vision0_pts/len,0.0370824,0.0370824,0.0370824,0.0370824,0.0370824,0.0485736,0.0485736,0.0485736,0.0485736
vision0_pts_seg_norm,0.764758,0.764758,0.764758,0.764758,0.764758,0.764758,0.764758,0.764758,0.764758
vision0_pts/len_norm,0.106603,0.106603,0.106603,0.106603,0.106603,0.138865,0.138865,0.138865,0.138865
Double parking_count,1,1,1,1,1,5,5,5,5
Other Walking Issue_count,1,1,1,1,1,0,0,0,0


In [43]:
dc_vision0_details.head().T

,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
vision0_pts_seg,2,1,0,0,0
vision0_pts/len,0.0103667,0.00757945,0,0,0
vision0_pts_seg_norm,0.350379,0.221065,0,0,0
vision0_pts/len_norm,0.0301949,0.022107,0,0,0
Other Driving Issue_count,0,0,0,0,0
Blocking the bikebox_count,0,0,0,0,0


In [44]:
dc_vision0_details.sort('vision0_pts/len')

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,OBJECTID,STREETSEGID,SHAPE_Length,geometry,vision0_pts_seg,vision0_pts/len,vision0_pts_seg_norm,vision0_pts/len_norm,Other Driving Issue_count,Blocking the bikebox_count,...,Red light running_count/len_norm,Blocking the crosswalk_count/len,Blocking the crosswalk_count_norm,Blocking the crosswalk_count/len_norm,Jaywalking_count/len,Jaywalking_count_norm,Jaywalking_count/len_norm,Not enough time to cross_count/len,Not enough time to cross_count_norm,Not enough time to cross_count/len_norm
7535,6614,915,203.986987,LINESTRING (-76.92339934318508 38.893053516052...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9604,8447,701,57.458678,LINESTRING (-76.99950558752809 38.925593529412...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9605,8448,1490,44.826205,LINESTRING (-76.98559753548959 38.924574160428...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9606,8449,6863,98.386494,LINESTRING (-76.98530598890108 38.924243791289...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9608,8451,4353,253.321834,LINESTRING (-76.98305713331489 38.922694505991...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9609,8452,8182,181.299154,LINESTRING (-76.98086440794096 38.921186714542...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9610,8453,9370,33.719677,LINESTRING (-76.97929535926961 38.920107393708...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9611,8454,599,144.321725,LINESTRING (-76.96550816309772 38.926767719994...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9613,8456,8206,97.932269,LINESTRING (-77.00746095793195 38.878414725769...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9614,8457,2814,115.641385,LINESTRING (-77.00745798452517 38.877532533818...,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [124]:
#dc_vision0_details = dc_vision0_details.drop('geometry', axis=1)
dc_vision0_details = dc_vision0_details.drop('Unnamed: 0', axis=1)
dc_vision0_details.head().T

,0,1,2,3,4
OBJECTID,1.000000,2.000000,3.000000,4.000000,5.000000
STREETSEGID,876.000000,13161.000000,8240.000000,10395.000000,2216.000000
SHAPE_Length,192.925131,131.935677,117.887011,108.331118,119.190312
vision0_pts_seg,2.000000,1.000000,0.000000,0.000000,0.000000
vision0_pts/len,0.010367,0.007579,0.000000,0.000000,0.000000
vision0_pts_seg_norm,0.350379,0.221065,0.000000,0.000000,0.000000
vision0_pts/len_norm,0.030195,0.022107,0.000000,0.000000,0.000000
Red light running_count,0.000000,0.000000,0.000000,0.000000,0.000000
Accessibility Issue_count,0.000000,0.000000,0.000000,0.000000,0.000000
Failure to stop for pedestrians_count,0.000000,0.000000,0.000000,0.000000,0.000000


In [125]:
dc_vision0_details.to_csv('../DC_Vision_Zero_Details.csv')

In [ ]:
def gradient_color(percent):
    min_color = np.array([255,255,255])
    max_color = np.array([248,105,107])
    return '#%02x%02x%02x' % tuple([int(k) for k in min_color+(max_color-min_color)*percent])

def write_var(col_name, var_name, df,f):
    df['color']=df[col_name].apply(gradient_color)
    f.write('var %s = %s;\n' % (var_name, df.to_json()))

with open('../visualization/Volumes of Vision0_new.js','w+') as f:
    write_var('vision0_pts_seg_norm', 'vision0_ptseg', dc_vision0_details,f)
    write_var('vision0_pts/len_norm', 'vision0_ptseg_norm', dc_vision0_details,f)

In [1]:
import geopandas as gp
from shapely.geometry import LineString, Point
import rtree
import os
import numpy as np
from itertools import chain

In [2]:
from pandas import DataFrame
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [3]:
data = DataFrame.from_csv("../Cityworks_Service_Requests_filtered.csv", index_col=False)
#data_drop = data.dropna(subset = ['X', 'Y']);
points = [Point(row['X'], row['Y']) for key, row in data.iterrows()]

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (15,23,26,32,40) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
geo_df = GeoDataFrame(data,geometry=points)

In [5]:
geo_df.shape

(204924, 53)

In [6]:
geo_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 204924 entries, 0 to 204923
Data columns (total 53 columns):
X                       204924 non-null float64
Y                       204924 non-null float64
OBJECTID                204924 non-null int64
REQUESTID               204924 non-null int64
WORKORDERID             76541 non-null float64
CSRNUMBER               204468 non-null object
DESCRIPTION             204924 non-null object
STATUS                  204906 non-null object
REQUESTCATEGORY         201914 non-null object
INITIATEDDATE           204912 non-null object
CLOSEDDATE              172390 non-null object
INSPECTIONDATE          176017 non-null object
INSPECTIONCOMPLETE      155067 non-null object
SUBMITTEDTODATE         86762 non-null object
DISPATCHEDTODATE        58197 non-null object
CANCELEDDATE            57 non-null object
PRIORITY                199304 non-null float64
INITIATEDBY             204924 non-null object
SUBMITTEDTO             86766 non-null o

In [8]:
fname = "../Cityworks_Service_Requests_complete.geojson"
geo_df.to_file(fname, driver='GeoJSON')

In [66]:
data = DataFrame.from_csv("../Cityworks_Service_Requests_Parking_Meter.csv", index_col=False)
#data_drop = data.dropna(subset = ['X', 'Y']);
points = [Point(row['X'], row['Y']) for key, row in data.iterrows()]

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (12,13,14,18,19,21,35,40) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [68]:
geo_df = GeoDataFrame(data,geometry=points)
geo_df.shape

(455100, 53)

In [69]:
fname = "../Cityworks_Service_Requests_Parking_Meter.geojson"
geo_df.to_file(fname, driver='GeoJSON')

In [4]:
city_req_pts = gp.read_file("../Cityworks_Service_Requests_filtered.geojson")

In [46]:
city_req_pts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 204924 entries, 0 to 204923
Data columns (total 53 columns):
ADDRESS                 204898 non-null object
ANC                     204438 non-null object
AWI                     5360 non-null object
BID                     23290 non-null object
CANCELEDBY              57 non-null object
CANCELEDDATE            57 non-null object
CFAR                    5044 non-null object
CLOSEDBY                172390 non-null object
CLOSEDDATE              172390 non-null object
CSRNUMBER               204468 non-null object
DAYSTOCLOSE             204924 non-null int64
DAYSTOINSPECT           176017 non-null float64
DESCRIPTION             204924 non-null object
DISPATCHEDTO            58146 non-null object
DISPATCHEDTODATE        58197 non-null object
EDZ                     9898 non-null object
FISCALYEAR              204924 non-null int64
HISTORICDISTRICT        43276 non-null object
INITIATEDBY             204924 non-null object
INITIAT

In [5]:
dc_segments = gp.read_file("../dc-maps-master/maps/street-segments.geojson")

In [6]:
city_req_pts_req_pts.head().T

,0,1,2,3,4
ADDRESS,1773 WILLARD STREET NW,SHERIDAN ST & 5TH STREET NW,1801 H STREET NE,7510 17TH STREET NW,5600 - 5667 BLOCK OF 2ND STREET NW
ANC,3D,4B,5D,4A,4B
AWI,None,None,None,None,None
BID,None,None,None,None,None
CANCELEDBY,None,None,None,None,None
CANCELEDDATE,None,None,None,None,None
CFAR,None,None,None,None,None
CLOSEDBY,"HILL, GREER A","LATTIMORE, JELANI","LATTIMORE, JELANI",None,"MARTIN, KIANA"
CLOSEDDATE,2013-07-21T15:01:41,2014-12-13T14:52:48,2014-12-13T14:40:07,None,2014-08-09T12:15:24
CSRNUMBER,13-00079017,10-00188633,09-03080547,09-03124614,10-00155132


In [7]:
print(city_req_pts[city_req_pts['INITIATEDDATE'].str.contains('2013', na=False)].shape[0])
print(city_req_pts[city_req_pts['INITIATEDDATE'].str.contains('2014', na=False)].shape[0])
print(city_req_pts[city_req_pts['INITIATEDDATE'].str.contains('2015', na=False)].shape[0])
print(city_req_pts[city_req_pts['INITIATEDDATE'].str.contains('2016', na=False)].shape[0])

32926
46080
49852
40918


In [48]:
dc_seg_buffer_gp = dc_segments[['OBJECTID','STREETSEGID','geometry']].copy()
dc_seg_buffer_gp.geometry = dc_seg_buffer_gp.buffer(0.0001)
city_req_buffer_gp = city_req_pts[['OBJECTID','DESCRIPTION','geometry','INITIATEDDATE','ADDRESS']].copy()
city_req_buffer_gp.geometry = city_req_buffer_gp.buffer(0.0001)
joined = sjoin(city_req_buffer_gp, dc_seg_buffer_gp, how='left', op='intersects')
#gb = joined.reset_index().groupby('OBJECTID_LEFT').count()
    
#print(gb[gb.OBJECTID_right==0].shape[0], ' points spatially match no segment')
#print(gb[gb.OBJECTID_right==1].shape[0], ' points spatially match 1 segment')
#print(gb[gb.OBJECTID_right>1].shape[0], ' points spatially match multiple segments')

In [1]:
import geopandas as gp
from shapely.geometry import LineString, Point
import rtree
import os
import numpy as np

In [2]:
from geopandas.tools import sjoin
import pandas as pd
from IPython.display import display
import snap

In [3]:
from itertools import chain

In [49]:
joined.head().T

,0,1,1,1,1
OBJECTID_left,1,2,2,2,2
DESCRIPTION,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR
geometry,"POLYGON ((-77.08052689 38.90750167, -77.080527...","POLYGON ((-77.01980087999999 38.96614244, -77....","POLYGON ((-77.01980087999999 38.96614244, -77....","POLYGON ((-77.01980087999999 38.96614244, -77....","POLYGON ((-77.01980087999999 38.96614244, -77...."
INITIATEDDATE,2013-04-09T15:29:30,2014-12-13T16:13:19,2014-12-13T16:13:19,2014-12-13T16:13:19,2014-12-13T16:13:19
ADDRESS,1773 WILLARD STREET NW,SHERIDAN ST & 5TH STREET NW,SHERIDAN ST & 5TH STREET NW,SHERIDAN ST & 5TH STREET NW,SHERIDAN ST & 5TH STREET NW
index_right,1742,3991,318,319,3990
OBJECTID_right,1743,3992,319,320,3991
STREETSEGID,7076,3338,2333,143,6957


In [50]:
gb = joined.reset_index().groupby('OBJECTID_left').count()

In [53]:
print(gb[gb.OBJECTID_right==0].shape[0], ' points spatially match no segment')
print(gb[gb.OBJECTID_right==1].shape[0], ' points spatially match 1 segment')
print(gb[gb.OBJECTID_right>1].shape[0], ' points spatially match multiple segments')

2589  points spatially match no segment
124413  points spatially match 1 segment
77922  points spatially match multiple segments


In [51]:
joined.shape

(408041, 8)

In [52]:
dc_segs_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]
dc_segs_statistic = dc_segs_statistic.merge(joined, left_on = 'STREETSEGID', right_index=True, how='left')

In [35]:
dc_segs_statistic.head().T

,0,1,2,2,2
STREETSEGID,876,13161,8240,8240,8240
OBJECTID,1,2,3,3,3
STREETSEGID_x,876,13161,8240,8240,8240
SHAPE_Length,192.925,131.936,117.887,117.887,117.887
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...
OBJECTID_left,7458,119270,76076,76076,76076
DESCRIPTION,SIDEWALK REPAIR,POTHOLES,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR
geometry_y,"POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.06395619 38.94511063, -77.063956...","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77...."
INITIATEDDATE,2011-10-12T16:34:55,2012-04-26T09:56:42,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24
ADDRESS,1100 - 1199 BLOCK OF I STREET NW,4339 CONNECTICUT AVENUE NW,945010,945010,945010


In [21]:
dc_segs_statistic.shape

(24437, 13)

In [22]:
city_req_pts.shape

(204924, 53)

In [36]:
dc_segs_statistic.groupby('OBJECTID_left').head().T

,0,1,2,2,2,2,3,4,5,6,...,13518,13518,13518,13519,13519,13519,13520,13520,13520,13521
STREETSEGID,876,13161,8240,8240,8240,8240,10395,2216,10090,8384,...,15306,15306,15306,15284,15284,15284,15307,15307,15307,15247
OBJECTID,1,2,3,3,3,3,4,5,6,7,...,13519,13519,13519,13520,13520,13520,13521,13521,13521,13522
STREETSEGID_x,876,13161,8240,8240,8240,8240,10395,2216,10090,8384,...,15306,15306,15306,15284,15284,15284,15307,15307,15307,15247
SHAPE_Length,192.925,131.936,117.887,117.887,117.887,117.887,108.331,119.19,119.101,78.8781,...,107.921,107.921,107.921,96.5238,96.5238,96.5238,53.2962,53.2962,53.2962,40.6968
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...,LINESTRING (-77.0116172263626 38.9576690437764...,LINESTRING (-77.0116124400554 38.9587419287958...,...,LINESTRING (-77.01885842791256 38.836994168269...,LINESTRING (-77.01885842791256 38.836994168269...,LINESTRING (-77.01885842791256 38.836994168269...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01826374709646 38.822439566938...
OBJECTID_left,7458,119270,76076,76076,76076,76076,97451,21321,94294,79465,...,135096,135096,135096,133869,133869,133869,135098,135098,135098,133453
DESCRIPTION,SIDEWALK REPAIR,POTHOLES,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR,ALLEYLIGHT REPAIR,TREE REMOVAL,STREETLIGHT REPAIR,BICYCLES,...,SIGN REPLACE,SIGN REPLACE,SIGN REPLACE,TRAFFIC SIGNAL MAIN,TRAFFIC SIGNAL MAIN,TRAFFIC SIGNAL MAIN,STREETLIGHT REPAIR,STREETLIGHT REPAIR,STREETLIGHT REPAIR,SPEED STUDY DATA MAP
geometry_y,"POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.06395619 38.94511063, -77.063956...","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.07509421 38.95903292, -77.075094...","POLYGON ((-77.07066087 38.96310065, -77.070661...","POLYGON ((-77.01512099999999 38.89907316, -77....","POLYGON ((-77.02204698999999 38.90565322, -77....",...,"POLYGON ((-77.02131177999999 38.93770468, -77....","POLYGON ((-77.02131177999999 38.93770468, -77....","POLYGON ((-77.02131177999999 38.93770468, -77....","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-77.01769534 38.90432147, -77.017695..."
INITIATEDDATE,2011-10-12T16:34:55,2012-04-26T09:56:42,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-03-20T20:08:29,2011-11-01T11:16:59,2012-03-15T14:23:36,2012-02-16T11:41:47,...,2012-05-22T12:29:28,2012-05-22T12:29:28,2012-05-22T12:29:28,2012-05-19T16:02:56,2012-05-19T16:02:56,2012-05-19T16:02:56,2012-05-22T12:29:30,2012-05-22T12:29:30,2012-05-22T12:29:30,2012-05-18T19:53:42
ADDRESS,1100 - 1199 BLOCK OF I STREET NW,4339 CONNECTICUT AVENUE NW,945010,945010,945010,945010,3803 JENIFER STREET NW,3614 LIVINGSTON STREET NW,701 - 727 BLOCK OF 3RD STREET NW,700 M STREET NW,...,7TH STREET NW AND QUINCY STREET NW,7TH STREET NW AND QUINCY STREET NW,7TH STREET NW AND QUINCY STREET NW,4801 14TH STREET NW,4801 14TH STREET NW,4801 14TH STREET NW,49TH STREET SE AND AYERS PLACE SE,49TH STREET SE

In [54]:
dc_segs_statistic = dc_segs_statistic.merge(joined, left_on = 'STREETSEGID', right_index=True, how='left')

In [55]:
dc_segs_statistic.shape

(66177, 22)

In [38]:
dc_segs_statistic.head().T

,0,1,2,2,2
STREETSEGID,876,13161,8240,8240,8240
STREETSEGID_x,876,13161,8240,8240,8240
OBJECTID,1,2,3,3,3
STREETSEGID_x,876,13161,8240,8240,8240
SHAPE_Length,192.925,131.936,117.887,117.887,117.887
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...
OBJECTID_left_x,7458,119270,76076,76076,76076
DESCRIPTION_x,SIDEWALK REPAIR,POTHOLES,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR
geometry_y,"POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.06395619 38.94511063, -77.063956...","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77...."
INITIATEDDATE_x,2011-10-12T16:34:55,2012-04-26T09:56:42,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24


In [39]:
dc_segs_statistic.groupby('STREETSEGID').head().T

,0,1,2,2,2,2,2,3,4,5,...,13519,13519,13519,13519,13520,13520,13520,13520,13520,13521
STREETSEGID,876,13161,8240,8240,8240,8240,8240,10395,2216,10090,...,15284,15284,15284,15284,15307,15307,15307,15307,15307,15247
STREETSEGID_x,876,13161,8240,8240,8240,8240,8240,10395,2216,10090,...,15284,15284,15284,15284,15307,15307,15307,15307,15307,15247
OBJECTID,1,2,3,3,3,3,3,4,5,6,...,13520,13520,13520,13520,13521,13521,13521,13521,13521,13522
STREETSEGID_x,876,13161,8240,8240,8240,8240,8240,10395,2216,10090,...,15284,15284,15284,15284,15307,15307,15307,15307,15307,15247
SHAPE_Length,192.925,131.936,117.887,117.887,117.887,117.887,117.887,108.331,119.19,119.101,...,96.5238,96.5238,96.5238,96.5238,53.2962,53.2962,53.2962,53.2962,53.2962,40.6968
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...,LINESTRING (-77.0116172263626 38.9576690437764...,...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01826374709646 38.822439566938...
OBJECTID_left_x,7458,119270,76076,76076,76076,76076,76076,97451,21321,94294,...,133869,133869,133869,133869,135098,135098,135098,135098,135098,133453
DESCRIPTION_x,SIDEWALK REPAIR,POTHOLES,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR,SIDEWALK REPAIR,ALLEYLIGHT REPAIR,TREE REMOVAL,STREETLIGHT REPAIR,...,TRAFFIC SIGNAL MAIN,TRAFFIC SIGNAL MAIN,TRAFFIC SIGNAL MAIN,TRAFFIC SIGNAL MAIN,STREETLIGHT REPAIR,STREETLIGHT REPAIR,STREETLIGHT REPAIR,STREETLIGHT REPAIR,STREETLIGHT REPAIR,SPEED STUDY DATA MAP
geometry_y,"POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.06395619 38.94511063, -77.063956...","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.07509421 38.95903292, -77.075094...","POLYGON ((-77.07066087 38.96310065, -77.070661...","POLYGON ((-77.01512099999999 38.89907316, -77....",...,"POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-77.01769534 38.90432147, -77.017695..."
INITIATEDDATE_x,2011-10-12T16:34:55,2012-04-26T09:56:42,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-03-20T20:08:29,2011-11-01T11:16:59,2012-03-15T14:23:36,...,2012-05-19T16:02:56,2012-05-19T16:02:56,2012-05-19T16:02:56,2012-05-19T16:02:56,2012-05-22T12:29:30,2012-05-22T12:29:30,2012-05-22T12:29:30,2012-05-22T12:29:30,2012-05-22T12:29:30,2012-05-18T19:53:42


In [56]:
dc_city_req_stats = dc_segs_statistic[['STREETSEGID', 'OBJECTID']]

In [57]:
dc_city_req_stats.shape

(66177, 2)

In [29]:
city_req_pts.head().T

,0,1,2,3,4
ADDRESS,1773 WILLARD STREET NW,SHERIDAN ST & 5TH STREET NW,1801 H STREET NE,7510 17TH STREET NW,5600 - 5667 BLOCK OF 2ND STREET NW
ANC,3D,4B,5D,4A,4B
AWI,None,None,None,None,None
BID,None,None,None,None,None
CANCELEDBY,None,None,None,None,None
CANCELEDDATE,None,None,None,None,None
CFAR,None,None,None,None,None
CLOSEDBY,"HILL, GREER A","LATTIMORE, JELANI","LATTIMORE, JELANI",None,"MARTIN, KIANA"
CLOSEDDATE,2013-07-21T15:01:41,2014-12-13T14:52:48,2014-12-13T14:40:07,None,2014-08-09T12:15:24
CSRNUMBER,13-00079017,10-00188633,09-03080547,09-03124614,10-00155132


In [58]:
unique_req_categories = set()
for category in city_req_pts.DESCRIPTION:
    unique_req_categories.add(category)
print(len(unique_req_categories), unique_req_categories)

58 {'BICYCLES', 'SIGNS CONFLICTING', 'LIGHT-INFRASTRUCTURE', 'PUBLIC SPACE - OBSTR', 'BUS AND/OR RAIL ISSU', 'PARKING - HANDICAP', 'FLASHER MALFUNCTION', 'TRAFFIC SIGNAL MAIN', 'TRAFFIC CAMERA LOCAT', 'STREET REPAVING', 'POTHOLES', 'STREET REPAIR', 'CURB GUTTER REPAIR', 'MARKING MAINTENANCE', 'SIGN NEW', 'TREE REMOVAL', 'UTILITY REPAIR', 'DDOT CITATION', 'S0381', 'LIGHT-LIGHT POLE', 'SPEED STUDY DATA MAP', 'PEDESTRIAN SIGNAL RE', 'RESIDENT PARKING PER', 'SCHOOL CROSSING GUAR', 'SIGN REPLACE', 'PUSH BUTTON MALFUNCT', 'TREE PRUNING', 'TRAFFIC CALMING', 'LIGHT-OHGS', 'SIGN REMOVAL', 'ALLEY REPAIR', 'SIDEWALK REPAIR', 'S0499', 'MARKING INSTALLATION', 'STREETLIGHT REPAIR', 'BRIDGE MAINTENANCE R', 'S0460', 'ALLEYLIGHT REPAIR', 'MARKING MODIFICATION', 'MARKING REMOVAL', 'SIGN TRAFFIC CONTROL', 'PAVE', 'STRCAR', 'STREET PAVING SCHEDU', 'S0306', 'UTILITY REPAIR MAJOR', 'RESIDENTIAL PERMIT -', 'SAFE ROUTES TO SCHOO', 'SIDEWALK NEW', 'PEDESTRIAN SIGNAL MA', 'SIGN MISSING', 'BULB OUT', 'PEDESTRIAN

In [59]:
city_req_pt_seg = dc_segs_statistic[['STREETSEGID', 'OBJECTID', 'DESCRIPTION_x']]
city_req_pt_seg.columns = ['STREETSEGID', 'OBJECTID', 'DESCRIPTION']
city_req_typemerge = pd.DataFrame(city_req_pt_seg.groupby(['STREETSEGID', 'DESCRIPTION']).size().reset_index())

In [60]:
city_req_seg_counts = pd.DataFrame(city_req_pt_seg.groupby(['STREETSEGID']).size().reset_index())
city_req_seg_counts.columns = ['STREETSEGID', 'city_req_seg_count']

In [61]:
city_req_seg_counts.head().T

,0,1,2,3,4
STREETSEGID,1,2,3,4,5
city_req_seg_count,16,16,1,1,1


In [62]:
city_req_seg_stats = city_req_typemerge[['STREETSEGID']]

In [63]:
city_req_typemerge.columns = ['STREETSEGID', 'DESCRIPTION', 'count']
city_req_typemerge.head().T

,0,1,2,3,4
STREETSEGID,1,2,3,4,5
DESCRIPTION,SIDEWALK REPAIR,SIDEWALK REPAIR,CURB GUTTER REPAIR,CURB GUTTER REPAIR,SIDEWALK REPAIR
count,16,16,1,1,1


In [64]:
for ucategory in unique_req_categories:
    if(ucategory is not None):
        req_cat_data = city_req_typemerge[city_req_typemerge.DESCRIPTION==ucategory]
        ucategory_count = ucategory + '_count'
        req_cat_count = req_cat_data[['STREETSEGID', 'count']]
        req_cat_count.columns = ['STREETSEGID', ucategory_count]
        city_req_seg_stats = city_req_seg_stats.merge(req_cat_count, left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=False, how='left')

In [65]:
city_req_details = pd.DataFrame(city_req_seg_stats.fillna(0))
city_req_details = city_req_seg_counts.merge(city_req_details, left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=False, how='left')
city_req_details.head().T

,0,1,2,3,4
STREETSEGID,1.0,2.0,3.0,4.0,5.0
city_req_seg_count,16.0,16.0,1.0,1.0,1.0
BICYCLES_count,0.0,0.0,0.0,0.0,0.0
SIGNS CONFLICTING_count,0.0,0.0,0.0,0.0,0.0
LIGHT-INFRASTRUCTURE_count,0.0,0.0,0.0,0.0,0.0
PUBLIC SPACE - OBSTR_count,0.0,0.0,0.0,0.0,0.0
BUS AND/OR RAIL ISSU_count,0.0,0.0,0.0,0.0,0.0
PARKING - HANDICAP_count,0.0,0.0,0.0,0.0,0.0
FLASHER MALFUNCTION_count,0.0,0.0,0.0,0.0,0.0
TRAFFIC SIGNAL MAIN_count,0.0,0.0,0.0,0.0,0.0


In [66]:
dc_segs_statistic[dc_segs_statistic.STREETSEGID==1]

,STREETSEGID,STREETSEGID_x,OBJECTID,STREETSEGID_x,SHAPE_Length,geometry_x,OBJECTID_left_x,DESCRIPTION_x,geometry_y,INITIATEDDATE_x,...,OBJECTID_right_x,STREETSEGID_y,OBJECTID_left_y,DESCRIPTION_y,geometry,INITIATEDDATE_y,ADDRESS_y,index_right_y,OBJECTID_right_y,STREETSEGID_y
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,3992.0,3338.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3991.0,3992.0,3338.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,3992.0,3338.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,318.0,319.0,2333.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,3992.0,3338.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,319.0,320.0,143.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,3992.0,3338.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3990.0,3991.0,6957.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,319.0,2333.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3991.0,3992.0,3338.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,319.0,2333.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,318.0,319.0,2333.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,319.0,2333.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,319.0,320.0,143.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,319.0,2333.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3990.0,3991.0,6957.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,320.0,143.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3991.0,3992.0,3338.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,...,320.0,143.0,2,SIDEWALK REPAIR,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,318.0,319.0,2333.0


In [67]:
city_req_seg_counts.shape

(13522, 2)

In [68]:
city_req_seg_counts

,STREETSEGID,city_req_seg_count
0,1,16
1,2,16
2,3,1
3,4,1
4,5,1
5,6,16
6,7,1
7,8,16
8,9,1
9,10,1


In [69]:
dc_city_req_details = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]
dc_city_req_details = dc_city_req_details.merge(city_req_details, left_on = 'STREETSEGID', right_on='STREETSEGID', right_index=False, how='left')

In [70]:
dc_city_req_details.shape

(13522, 63)

In [71]:
dc_city_req_details['city_req_count/len'] = dc_city_req_details.city_req_seg_count/dc_city_req_details.SHAPE_Length

In [72]:
dc_city_req_details.head().T

,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
city_req_seg_count,1,1,16,1,1
BICYCLES_count,0,0,0,0,0
SIGNS CONFLICTING_count,0,0,0,0,0
LIGHT-INFRASTRUCTURE_count,0,0,0,0,0
PUBLIC SPACE - OBSTR_count,0,0,0,0,0
BUS AND/OR RAIL ISSU_count,0,0,0,0,0


In [73]:
for ucategory in unique_req_categories:
    if(ucategory is not None):
        ucategory_count = ucategory + '_count'
        ucategory_countlen = ucategory + '_count/len'
        dc_city_req_details[ucategory_countlen] = dc_city_req_details[ucategory_count]/dc_city_req_details.SHAPE_Length

In [76]:
for ucategory in unique_req_categories:
    if(ucategory is not None):
        ucategory_count = ucategory + '_count'
        ucategory_countlen = ucategory + '_count/len'
        normalize(ucategory_count, dc_city_req_details)
        normalize(ucategory_countlen, dc_city_req_details)

In [77]:
normalize('city_req_seg_count', dc_city_req_details)
normalize('city_req_count/len', dc_city_req_details)

In [78]:
dc_city_req_details = pd.DataFrame(dc_city_req_details.fillna(0))

In [79]:
dc_city_req_details.head().T

,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
city_req_seg_count,1,1,16,1,1
BICYCLES_count,0,0,0,0,0
SIGNS CONFLICTING_count,0,0,0,0,0
LIGHT-INFRASTRUCTURE_count,0,0,0,0,0
PUBLIC SPACE - OBSTR_count,0,0,0,0,0
BUS AND/OR RAIL ISSU_count,0,0,0,0,0


In [65]:
dc_city_req_details[dc_city_req_details.city_req_seg_count==16]

,OBJECTID,STREETSEGID,SHAPE_Length,geometry,city_req_seg_count,STREET PAVING SCHEDU_count,LIGHT-TUNNEL/UNDPASS_count,RESIDENTIAL PERMIT -_count,PUBLIC SPACE - OBSTR_count,SIGN NEW_count,...,PUSH BUTTON MALFUNCT_count_norm,PUSH BUTTON MALFUNCT_count/len_norm,SIGN TRAFFIC CONTROL_count_norm,SIGN TRAFFIC CONTROL_count/len_norm,PEDESTRIAN SIGNAL RE_count_norm,PEDESTRIAN SIGNAL RE_count/len_norm,TRAFFIC SIGNAL MAIN_count_norm,TRAFFIC SIGNAL MAIN_count/len_norm,city_req_seg_count_norm,city_req_count/len_norm
2,3,8240,117.887011,LINESTRING (-77.01157276320811 38.954558133776...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.64293,0.124189
8,9,7886,95.591087,LINESTRING (-77.01163025939501 38.969130242751...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.724232,0.155950,0.64293,0.151016
9,10,5660,157.511599,LINESTRING (-77.01166931573439 38.969983769179...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.64293,0.094404
10,11,9116,124.980589,LINESTRING (-77.01033664424322 38.904550094214...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.64293,0.117548
11,12,8292,121.299097,LINESTRING (-77.0100561260908 38.9555132405171...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.64293,0.120904
12,13,7297,119.131391,LINESTRING (-77.01261926029325 38.957669570480...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.64293,0.122971
15,16,10385,148.683739,LINESTRING (-77.01364139249108 38.892090605595...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.64293,0.099732
16,17,2902,126.698028,LINESTRING (-77.01324339541158 38.951215745942...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.64293,0.116046
18,19,9793,129.600852,LINESTRING (-77.01351167268689 38.953330819290...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.64293,0.113592
21,22,9806,34.121781,LINESTRING (-77.01364744974134 38.956284074045...,16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.64293,0.375214


In [64]:
dc_city_req_details.to_csv('../DC_City_Request_Details_filtered.csv')

In [ ]:
def gradient_color(percent):
    min_color = np.array([255,255,255])
    max_color = np.array([178,34,34])
    return '#%02x%02x%02x' % tuple([int(k) for k in min_color+(max_color-min_color)*percent])

def write_var(col_name, var_name, df,f):
    df['color']=df[col_name].apply(gradient_color)
    f.write('var %s = %s;\n' % (var_name, df.to_json()))

with open('../visualization/Volumes of City Requests.js','w+') as f:
    write_var('city_req_seg_count_norm', 'city_req_count', dc_city_req_details,f)
    write_var('city_req_count/len_norm', 'city_req_count_norm', dc_city_req_details,f)

In [116]:
dc_seg_city_req_statistic_drop[dc_seg_city_req_statistic_drop.STREETSEGID_x==6316]

,OBJECTID,STREETSEGID_x,SHAPE_Length,geometry,STREETSEGID_y,city_req_seg_count,city_req/len,city_req_seg_count_norm,city_req/len_norm,color
12898,12899,6316,661.947921,LINESTRING (-77.03665083891831 38.886935862966...,6477.0,1.0,0.001511,0.157293,0.001202,#fefefe


In [117]:
dc_seg_city_req_statistic_drop[dc_seg_city_req_statistic_drop.STREETSEGID_y==6316]

,OBJECTID,STREETSEGID_x,SHAPE_Length,geometry,STREETSEGID_y,city_req_seg_count,city_req/len,city_req_seg_count_norm,city_req/len_norm,color
8696,8697,6160,97.850095,LINESTRING (-77.00204275692717 38.879293174632...,6316.0,16.0,0.163515,0.64293,0.120608,#f5e4e4


In [120]:
dc_seg_city_req_statistic_drop.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 12984 entries, 0 to 13273
Data columns (total 10 columns):
OBJECTID                   12984 non-null int64
STREETSEGID_x              12984 non-null int64
SHAPE_Length               12984 non-null float64
geometry                   12984 non-null object
STREETSEGID_y              12984 non-null float64
city_req_seg_count         12984 non-null float64
city_req/len               12984 non-null float64
city_req_seg_count_norm    12984 non-null float64
city_req/len_norm          12984 non-null float64
color                      12984 non-null object
dtypes: float64(6), int64(2), object(2)
memory usage: 1.1+ MB


In [121]:
city_req_pts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 204924 entries, 0 to 204923
Data columns (total 53 columns):
ADDRESS                 204898 non-null object
ANC                     204438 non-null object
AWI                     5360 non-null object
BID                     23290 non-null object
CANCELEDBY              57 non-null object
CANCELEDDATE            57 non-null object
CFAR                    5044 non-null object
CLOSEDBY                172390 non-null object
CLOSEDDATE              172390 non-null object
CSRNUMBER               204468 non-null object
DAYSTOCLOSE             204924 non-null int64
DAYSTOINSPECT           176017 non-null float64
DESCRIPTION             204924 non-null object
DISPATCHEDTO            58146 non-null object
DISPATCHEDTODATE        58197 non-null object
EDZ                     9898 non-null object
FISCALYEAR              204924 non-null int64
HISTORICDISTRICT        43276 non-null object
INITIATEDBY             204924 non-null object
INITIAT

In [124]:
city_counts = city_req_pts.groupby('DESCRIPTION').count().reset_index().sort()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  if __name__ == '__main__':


In [80]:
city_req_pm_pts = gp.read_file("../Cityworks_Service_Requests_Parking_Meter.geojson")
city_req_pm_pts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 455100 entries, 0 to 455099
Data columns (total 53 columns):
ADDRESS                 455100 non-null object
ANC                     455088 non-null object
AWI                     23115 non-null object
BID                     225073 non-null object
CANCELEDBY              0 non-null object
CANCELEDDATE            0 non-null object
CFAR                    162265 non-null object
CLOSEDBY                438986 non-null object
CLOSEDDATE              438986 non-null object
CSRNUMBER               455097 non-null object
DAYSTOCLOSE             455100 non-null int64
DAYSTOINSPECT           399260 non-null float64
DESCRIPTION             455100 non-null object
DISPATCHEDTO            13 non-null object
DISPATCHEDTODATE        19 non-null object
EDZ                     599 non-null object
FISCALYEAR              455100 non-null int64
HISTORICDISTRICT        174351 non-null object
INITIATEDBY             455100 non-null object
INITIATEDDA

In [83]:
city_req_pm_pts.head().T

,0,1,2,3,4
ADDRESS,1100 - 1199 BLOCK OF 7TH STREET NW,FRONTAGE ROAD SW AND 7TH STREET SW,7TH STREET NW AND H STREET NW,10TH STREET NW AND NEW YORK AVENUE NW,FRONTAGE ROAD SW AND 7TH STREET SW
ANC,2F,6D,2C,2C,6D
AWI,None,None,None,None,None
BID,Downtown BID,None,Downtown BID,Downtown BID,None
CANCELEDBY,None,None,None,None,None
CANCELEDDATE,None,None,None,None,None
CFAR,None,Shipstead Luce,None,None,Shipstead Luce
CLOSEDBY,ISLIMS,ISLIMS,ISLIMS,ISLIMS,ISLIMS
CLOSEDDATE,2011-10-08T05:02:06,2011-10-12T10:13:07,2011-10-01T07:14:21,2011-10-06T12:37:18,2011-10-12T10:15:33
CSRNUMBER,11-00310068,11-00310069,11-00310070,11-00310072,11-00310071


In [84]:
dc_seg_buffer_gp = dc_segments[['OBJECTID','STREETSEGID','geometry']].copy()
dc_seg_buffer_gp.geometry = dc_seg_buffer_gp.buffer(0.0001)
city_req_pm_buffer_gp = city_req_pm_pts[['OBJECTID','DESCRIPTION','geometry','INITIATEDDATE','ADDRESS']].copy()
city_req_pm_buffer_gp.geometry = city_req_pm_buffer_gp.buffer(0.0001)
joined = sjoin(city_req_pm_buffer_gp, dc_seg_buffer_gp, how='left', op='intersects')
gb = joined.reset_index().groupby('OBJECTID_left').count()
    
print(gb[gb.OBJECTID_right==0].shape[0], ' points spatially match no segment')
print(gb[gb.OBJECTID_right==1].shape[0], ' points spatially match 1 segment')
print(gb[gb.OBJECTID_right>1].shape[0], ' points spatially match multiple segments')

1043  points spatially match no segment
335120  points spatially match 1 segment
118937  points spatially match multiple segments


In [85]:
dc_city_req_stats.head().T

,0,1,2,2,2
STREETSEGID,876,13161,8240,8240,8240
OBJECTID,1,2,3,3,3


In [86]:
dc_segs_pm_statistic = dc_segs_statistic.merge(joined, left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=False, how='left')

In [87]:
dc_segs_pm_statistic.head().T

,0,1,2,3,4
STREETSEGID,876,876,13161,8240,8240
STREETSEGID_x,876,876,13161,8240,8240
OBJECTID,1,1,2,3,3
STREETSEGID_x,876,876,13161,8240,8240
SHAPE_Length,192.925,192.925,131.936,117.887,117.887
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...
OBJECTID_left_x,7458,7458,119270,76076,76076
DESCRIPTION_x,SIDEWALK REPAIR,SIDEWALK REPAIR,POTHOLES,SIDEWALK REPAIR,SIDEWALK REPAIR
geometry_y,"POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.06395619 38.94511063, -77.063956...","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77...."
INITIATEDDATE_x,2011-10-12T16:34:55,2011-10-12T16:34:55,2012-04-26T09:56:42,2012-02-09T16:14:24,2012-02-09T16:14:24


In [88]:
dc_city_req_pm_stats = dc_segs_pm_statistic[['STREETSEGID', 'OBJECTID']]
dc_city_req_pm_stats.head().T

,0,1,2,3,4
STREETSEGID,876,876,13161,8240,8240
OBJECTID,1,1,2,3,3


In [89]:
dc_city_req_pm_count = dc_city_req_pm_stats.groupby('STREETSEGID').size().reset_index()

In [90]:
dc_city_req_pm_count.columns = ['STREETSEGID', 'Parking Meter_count']

In [91]:
print(dc_city_req_pm_count.shape)
dc_city_req_pm_count.head().T

(13522, 2)


,0,1,2,3,4
STREETSEGID,1,2,3,4,5
Parking Meter_count,16,16,236,21,1


In [92]:
dc_city_req_details = dc_city_req_details.merge(dc_city_req_pm_count, left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=False, how='left')

In [93]:
dc_city_req_details = dc_city_req_details.fillna(0)

In [94]:
dc_city_req_details[['city_req_seg_count']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13522 entries, 0 to 13521
Data columns (total 1 columns):
city_req_seg_count    13522 non-null int64
dtypes: int64(1)
memory usage: 211.3 KB


In [95]:
dc_city_req_details['city_req_seg_count'] = dc_city_req_details['city_req_seg_count'] + dc_city_req_details['Parking Meter_count']

In [96]:
print(dc_city_req_details.shape)
dc_city_req_details.head().T

(13522, 241)


,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
city_req_seg_count,3,2,80,6,2
BICYCLES_count,0,0,0,0,0
SIGNS CONFLICTING_count,0,0,0,0,0
LIGHT-INFRASTRUCTURE_count,0,0,0,0,0
PUBLIC SPACE - OBSTR_count,0,0,0,0,0
BUS AND/OR RAIL ISSU_count,0,0,0,0,0


In [97]:
dc_city_req_details[dc_city_req_details['city_req_seg_count']>dc_city_req_details['Parking Meter_count']]

,OBJECTID,STREETSEGID,SHAPE_Length,geometry,city_req_seg_count,BICYCLES_count,SIGNS CONFLICTING_count,LIGHT-INFRASTRUCTURE_count,PUBLIC SPACE - OBSTR_count,BUS AND/OR RAIL ISSU_count,...,SCHOOL SUBSIDY_count/len_norm,CHILD SAFETY SEAT PR_count_norm,CHILD SAFETY SEAT PR_count/len_norm,S0457_count_norm,S0457_count/len_norm,FLASHER MODIFICATION_count_norm,FLASHER MODIFICATION_count/len_norm,city_req_seg_count_norm,city_req_count/len_norm,Parking Meter_count
0,1,876,192.925131,LINESTRING (-77.01239193659748 38.950836102255...,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157293,0.005045,2
1,2,13161,131.935677,LINESTRING (-77.01163764014346 38.953383268855...,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157293,0.007368,1
2,3,8240,117.887011,LINESTRING (-77.01157276320811 38.954558133776...,80,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.642930,0.124189,64
3,4,10395,108.331118,LINESTRING (-77.0116155077942 38.9556194835925...,6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157293,0.008966,5
4,5,2216,119.190312,LINESTRING (-77.01161670521323 38.956595348877...,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157293,0.008153,1
5,6,10090,119.101173,LINESTRING (-77.0116172263626 38.9576690437764...,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157293,0.008159,2
6,7,8384,78.878075,LINESTRING (-77.0116124400554 38.9587419287958...,3,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157293,0.012293,2
7,8,11625,79.410694,LINESTRING (-77.0116157864189 38.9594524093840...,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157293,0.012211,1
8,9,7886,95.591087,LINESTRING (-77.01163025939501 38.969130242751...,32,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.642930,0.151016,16
9,10,5660,157.511599,LINESTRING (-77.01166931573439 38.969983769179...,32,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.642930,0.094404,16


In [98]:
dc_city_req_details['Parking_Meter_count/len'] = dc_city_req_details['Parking Meter_count']/dc_city_req_details.SHAPE_Length
dc_city_req_details['city_req_count/len'] = dc_city_req_details['city_req_seg_count']/dc_city_req_details.SHAPE_Length

normalize('Parking Meter_count', dc_city_req_details)
normalize('Parking_Meter_count/len', dc_city_req_details)
normalize('city_req_seg_count', dc_city_req_details)
normalize('city_req_count/len', dc_city_req_details)

In [104]:
print(dc_city_req_details.shape)
dc_city_req_details.head().T

(13522, 244)


,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
city_req_seg_count,3,2,80,6,2
STREET PAVING SCHEDU_count,0,0,0,0,0
LIGHT-TUNNEL/UNDPASS_count,0,0,0,0,0
RESIDENTIAL PERMIT -_count,0,0,0,0,0
PUBLIC SPACE - OBSTR_count,0,0,0,0,0
SIGN NEW_count,0,0,0,0,0


In [106]:
dc_city_req_details.to_csv('../DC_City_Request_Details_complete.csv')

In [8]:
crashes = pd.read_csv('../Crashes_in_the_District_of_Columbia.csv')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4,5,6,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
print(crashes.shape)
crashes.head().T

(152744, 64)


,0,1,2,3,4
﻿X,-77.0512,-77.0199,-76.9764,-77.0275,-77.0398
Y,38.8972,38.957,38.874,38.9287,38.9
CRASHID,2.84879e+08,2.84879e+08,2.84794e+08,2.84794e+08,2.84794e+08
CRIMEID,2.84878e+08,2.84877e+08,2.84794e+08,2.84796e+08,2.84796e+08
ISREPORTONSCENE,1,1,1,1,1
WASMAJORCRASHNOTIFIED,0,0,0,0,0
MAJORCRASHNOTIFIEDDATE,NaN,NaN,NaN,NaN,NaN
MAJORCRASHNOTIFIEDPERSONID,null,null,null,null,null
SCHOOLBUSRELATED,null,null,null,null,null
ISJUNCTIONINTERCHANGEAREA,null,null,null,null,null


In [8]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152744 entries, 0 to 152743
Data columns (total 64 columns):
﻿X                              124278 non-null float64
Y                               124278 non-null float64
CRASHID                         152744 non-null float64
CRIMEID                         152744 non-null float64
ISREPORTONSCENE                 152744 non-null object
WASMAJORCRASHNOTIFIED           152744 non-null object
MAJORCRASHNOTIFIEDDATE          50 non-null object
MAJORCRASHNOTIFIEDPERSONID      152744 non-null object
SCHOOLBUSRELATED                152744 non-null object
ISJUNCTIONINTERCHANGEAREA       152744 non-null object
JUNCTIONSPECIFICLOCATION        152744 non-null object
INTERSECTIONTYPE                152744 non-null object
TRAFFICWAYRELATION              152744 non-null object
TRAFFICWAYRELATIONOTHER         152744 non-null object
ISWORKZONERELATED               152744 non-null int64
WORKZONETYPE                    152744 non-null object
WEREWORKZO

In [17]:
crash_csv = crashes[pd.isnull(crashes['XCOORD'])==0]
crash_csv.shape

(124278, 64)

In [18]:
crash_csv.head().T

,0,1,2,3,4
﻿X,-77.0512,-77.0199,-76.9764,-77.0275,-77.0398
Y,38.8972,38.957,38.874,38.9287,38.9
CRASHID,2.84879e+08,2.84879e+08,2.84794e+08,2.84794e+08,2.84794e+08
CRIMEID,2.84878e+08,2.84877e+08,2.84794e+08,2.84796e+08,2.84796e+08
ISREPORTONSCENE,1,1,1,1,1
WASMAJORCRASHNOTIFIED,0,0,0,0,0
MAJORCRASHNOTIFIEDDATE,NaN,NaN,NaN,NaN,NaN
MAJORCRASHNOTIFIEDPERSONID,null,null,null,null,null
SCHOOLBUSRELATED,null,null,null,null,null
ISJUNCTIONINTERCHANGEAREA,null,null,null,null,null


In [25]:
points = [Point(row[crash_csv.columns[0]], row['Y']) for key, row in crash_csv.iterrows()]
geo_df = GeoDataFrame(crash_csv,geometry=points)
geo_df.to_file('../Crashes_in_the_District_of_Columbia.geojson', driver='GeoJSON')

In [15]:
pd.isnull(crashes.XCOORD)

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9          True
10        False
11         True
12        False
13        False
14        False
15         True
16        False
17        False
18         True
19        False
20        False
21        False
22        False
23        False
24         True
25        False
26        False
27         True
28        False
29         True
          ...  
152714    False
152715    False
152716    False
152717    False
152718    False
152719    False
152720    False
152721    False
152722    False
152723    False
152724    False
152725    False
152726    False
152727    False
152728    False
152729     True
152730    False
152731    False
152732    False
152733     True
152734     True
152735    False
152736    False
152737     True
152738     True
152739    False
152740    False
152741     True
152742     True
152743     True
Name: XCOORD, dtype: boo

In [23]:
crash_csv.columns

Index(['﻿X', 'Y', 'CRASHID', 'CRIMEID', 'ISREPORTONSCENE',
       'WASMAJORCRASHNOTIFIED', 'MAJORCRASHNOTIFIEDDATE',
       'MAJORCRASHNOTIFIEDPERSONID', 'SCHOOLBUSRELATED',
       'ISJUNCTIONINTERCHANGEAREA', 'JUNCTIONSPECIFICLOCATION',
       'INTERSECTIONTYPE', 'TRAFFICWAYRELATION', 'TRAFFICWAYRELATIONOTHER',
       'ISWORKZONERELATED', 'WORKZONETYPE', 'WEREWORKZONEWORKERSPRESENT',
       'WORKZONELOEPRESENT', 'WORKZONELOCATION', 'STREETLIGHTING',
       'STREETLIGHTINGOTHER', 'POSTEDSPEEDLIMIT', 'FIRSTHARMFULEVENT',
       'FIRSTHARMFULEVENTOTHER', 'FIRSTHARMFULEVENTSPECIFICS',
       'FIRSTHARMFULEVENTIMPACT', 'FIRSTHARMFULEVENTRELATIVELOC',
       'NUMBERPHOTOSTAKEN', 'DCLOCATIONFEET', 'DCLOCATIONBEARING',
       'DCLOCATIONINTERSECTIONBLOCK', 'DCLOCATIONSECTION',
       'DCLOCATIONMILEPOST', 'DCLOCATIONPEPCOPOLLNO', 'DCLOCATIONEXITRAMP',
       'DCLOCATIONBRIDGE', 'DCLOCATIONTUNNEL', 'DCLOCATIONOTHER',
       'CRASHSCENEIMAGE', 'DWHSOURCE', 'SOURCEADDTIME', 'SOURCEMODTIME',
    

In [24]:
crash_csv[crash_csv.columns[0]]

0        -77.051173
1        -77.019915
2        -76.976363
3        -77.027512
4        -77.039808
5        -77.053672
6        -76.976223
7        -77.054235
8        -76.989948
10       -76.973277
12       -77.057246
13       -77.073952
14       -77.028031
16       -77.008282
17       -77.017549
19       -77.020908
20       -77.021623
21       -76.989788
22       -76.969524
23       -76.917393
25       -77.014947
26       -77.031522
28       -77.073483
31       -77.019903
32       -77.022809
33       -76.981266
34       -77.025127
36       -76.992450
37       -77.002040
38       -77.016206
            ...    
152701   -77.009044
152703   -77.055568
152704   -76.972162
152705   -77.039812
152706   -77.082658
152708   -77.032234
152710   -77.010759
152711   -77.002034
152714   -77.011404
152715   -77.026227
152716   -76.988859
152717   -77.078857
152718   -76.943392
152719   -77.003043
152720   -77.056700
152721   -76.942642
152722   -77.061345
152723   -76.993760
152724   -77.026490


In [8]:
crash_gpd = gp.read_file('../Crashes_in_the_District_of_Columbia.geojson')
print(crash_gpd.shape)
crash_gpd.head().T

(124278, 65)


,0,1,2,3,4
ADDRESS1,2400 VIRGINIA AVE NW,5500 BLOCK OF 5TH ST NW,YOUNG STREET & FAIRLAWN AVE SE,11TH & IRVING STREET NW,1700 H STREET
ADDRESS2,None,None,None,None,None
ADDRESS_ID,274046,802987,907447,902086,218406
CRASHEVENTTYPES,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION","COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION","COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION","COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION","COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION"
CRASHID,2.84879e+08,2.84879e+08,2.84794e+08,2.84794e+08,2.84794e+08
CRASHSCENEIMAGE,null,null,null,null,null
CRIMEID,2.84878e+08,2.84877e+08,2.84794e+08,2.84796e+08,2.84796e+08
CYCLISTSINVOLVED,None,None,None,None,None
DCLOCATIONBEARING,null,null,null,null,null
DCLOCATIONBRIDGE,null,null,null,null,null


In [9]:
print(crash_gpd[crash_gpd['REPORTDATE'].str.contains('2013', na=False)].shape[0])
print(crash_gpd[crash_gpd['REPORTDATE'].str.contains('2014', na=False)].shape[0])
print(crash_gpd[crash_gpd['REPORTDATE'].str.contains('2015', na=False)].shape[0])
print(crash_gpd[crash_gpd['REPORTDATE'].str.contains('2016', na=False)].shape[0])

16810
18983
19117
8469


In [5]:
dc_segments = gp.read_file("../dc-maps-master/maps/street-segments.geojson")

In [7]:
dc_seg_buffer_gp = dc_segments[['OBJECTID','STREETSEGID','geometry']].copy()
dc_seg_buffer_gp.geometry = dc_seg_buffer_gp.buffer(0.0001)
crash_buffer_gp = crash_gpd[['CRASHID','STREETSEGID','geometry']].dropna().copy()
crash_buffer_gp.geometry = crash_buffer_gp.buffer(0.0001)
joined = sjoin(crash_buffer_gp, dc_seg_buffer_gp, how='left', op='intersects')
gb = joined.reset_index().groupby('CRASHID').count()
    
print(gb[gb.STREETSEGID_right==0].shape[0], ' points spatially match no segment')
print(gb[gb.STREETSEGID_right==1].shape[0], ' points spatially match 1 segment')
print(gb[gb.STREETSEGID_right>1].shape[0], ' points spatially match multiple segments')

45575  points spatially match no segment
38870  points spatially match 1 segment
37909  points spatially match multiple segments


In [6]:
dc_seg_gp = dc_segments[['OBJECTID','STREETSEGID','geometry']].copy()
crash_drop_gp = crash_gpd[['CRASHID','STREETSEGID','geometry']].dropna().copy()

In [7]:
crash_merged = crash_drop_gp.merge(dc_seg_gp, left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=False, how='left')

In [49]:
dc_seg_buffer_gp.head().T

,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
geometry,POLYGON ((-77.01242844202709 38.95095383950174...,POLYGON ((-77.01164820900649 38.95361152346422...,POLYGON ((-77.01167643211649 38.95462238890561...,POLYGON ((-77.01171529267847 38.95571708362685...,POLYGON ((-77.01171706423393 38.95667960148032...


In [10]:
crash_merged.head().T

,0,1,2,3,4
CRASHID,2.84879e+08,2.84794e+08,2.84794e+08,2.84794e+08,2.84794e+08
STREETSEGID,2480.0,296.0,612.0,12926.0,628.0
geometry_x,POINT (-77.01991459279999 38.9569923951),POINT (-76.9763628826 38.8739935028),POINT (-77.0275123694 38.9287124837),POINT (-77.0398083253 38.8999813534),POINT (-77.0536723136 38.9034414121)
OBJECTID,NaN,NaN,NaN,NaN,NaN
geometry_y,NaN,NaN,NaN,NaN,NaN


In [35]:
gb = crash_merged.reset_index().groupby('CRASHID').count()
    
print(gb[gb.STREETSEGID_y==0].shape[0], ' points spatially match no segment')
print(gb[gb.STREETSEGID_y==1].shape[0], ' points spatially match 1 segment')
print(gb[gb.STREETSEGID_y>1].shape[0], ' points spatially match multiple segments')

122354  points spatially match no segment
0  points spatially match 1 segment
0  points spatially match multiple segments


In [37]:
crash_gpd[crash_gpd.STREETSEGID==876]

,ADDRESS1,ADDRESS2,ADDRESS_ID,CRASHEVENTTYPES,CRASHID,CRASHSCENEIMAGE,CRIMEID,CYCLISTSINVOLVED,DCLOCATIONBEARING,DCLOCATIONBRIDGE,...,WEATHER,WEREWORKZONEWORKERSPRESENT,WORKZONELOCATION,WORKZONELOEPRESENT,WORKZONETYPE,XCOORD,Y,YCOORD,geometry,﻿X


In [38]:
gb = joined.reset_index().groupby('CRASHID').count()
    
print(gb[gb.STREETSEGID_right==0].shape[0], ' points spatially match no segment')
print(gb[gb.STREETSEGID_right==1].shape[0], ' points spatially match 1 segment')
print(gb[gb.STREETSEGID_right>1].shape[0], ' points spatially match multiple segments')

45575  points spatially match no segment
38870  points spatially match 1 segment
37909  points spatially match multiple segments


In [42]:
gb[gb.STREETSEGID_right==0]

,index,STREETSEGID_left,geometry,index_right,OBJECTID,STREETSEGID_right
CRASHID,,,,,,
284558866.0,1,1,1,0,0,0
284558887.0,1,1,1,0,0,0
284558909.0,1,1,1,0,0,0
284558910.0,1,1,1,0,0,0
284559851.0,1,1,1,0,0,0
284560206.0,1,1,1,0,0,0
284560364.0,1,1,1,0,0,0
284560367.0,1,1,1,0,0,0
284560772.0,1,1,1,0,0,0


In [45]:
crash_gpd[crash_gpd.CRASHID==284558866].STREETSEGID

1634    959.0
Name: STREETSEGID, dtype: object

In [46]:
dc_segments[dc_segments.STREETSEGID==959]

,BEGINMEASURE,DIRECTIONALITY,ENDMEASURE,FACILITYID,FROMADDRESSLEFTTHEO,FROMADDRESSRIGHTTHEO,FROMNODEID,OBJECTID,OBJECTID_1,QUADRANT,...,SHAPE_Length,SOURCEID,STREETID,STREETSEGID,STREETTYPE,TOADDRESSLEFTTHEO,TOADDRESSRIGHTTHEO,TONODEID,UPDATETIMESTAMP,geometry
11101,0,2,120.33491,SEGID-959,4602.0,4559.0,13512,11102,10754,SE,...,120.334925,31534457,301534,959,RD,4612.0,4599.0,15911,2005-11-01T12:55:47,LINESTRING (-76.93660270520415 38.887584423568...


In [8]:
crash_merged[crash_merged.STREETSEGID==959]

,CRASHID,STREETSEGID,geometry_x,OBJECTID,geometry_y


In [56]:
crash_drop_gp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 122354 entries, 1 to 124277
Data columns (total 3 columns):
CRASHID        122354 non-null float64
STREETSEGID    122354 non-null object
geometry       122354 non-null object
dtypes: float64(1), object(2)
memory usage: 3.7+ MB


In [57]:
dc_seg_gp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13522 entries, 0 to 13521
Data columns (total 3 columns):
OBJECTID       13522 non-null int64
STREETSEGID    13522 non-null int64
geometry       13522 non-null object
dtypes: int64(2), object(1)
memory usage: 317.0+ KB


In [8]:
crash_drop_gp.STREETSEGID = crash_drop_gp.STREETSEGID.convert_objects(convert_numeric=True)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [12]:
crash_drop_gp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 122354 entries, 1 to 124277
Data columns (total 3 columns):
CRASHID        122354 non-null float64
STREETSEGID    122354 non-null float64
geometry       122354 non-null object
dtypes: float64(2), object(1)
memory usage: 3.7+ MB


In [9]:
crash_merged = crash_drop_gp.merge(dc_seg_gp, left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=True, how='left')

In [10]:
gb = crash_merged.reset_index().groupby('CRASHID').count()

print(gb[gb.STREETSEGID_y==0].shape[0], ' points spatially match no segment')
print(gb[gb.STREETSEGID_y==1].shape[0], ' points spatially match 1 segment')
print(gb[gb.STREETSEGID_y>1].shape[0], ' points spatially match multiple segments')

3861  points spatially match no segment
118493  points spatially match 1 segment
0  points spatially match multiple segments


In [11]:
crash_merged[crash_merged.STREETSEGID_x==959]

,CRASHID,STREETSEGID_x,geometry_x,OBJECTID,STREETSEGID_y,geometry_y
1634,284558866.0,959.0,POINT (-76.93658944560001 38.8869410377),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...
2162,284566606.0,959.0,POINT (-76.93670541289998 38.8871841467),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...
3594,285174243.0,959.0,POINT (-76.9362801435 38.8876503696),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...
5061,285490513.0,959.0,POINT (-76.93658944560001 38.8869410377),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...
8201,285790355.0,959.0,POINT (-76.9362801435 38.8876503696),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...
8405,285861785.0,959.0,POINT (-76.9362801435 38.8876503696),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...
11992,286057281.0,959.0,POINT (-76.9362801435 38.8876503696),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...
13957,286759330.0,959.0,POINT (-76.9362801435 38.8876503696),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...
17281,287151818.0,959.0,POINT (-76.9362801435 38.8876503696),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...
19426,287780418.0,959.0,POINT (-76.9362801435 38.8876503696),960.0,1015.0,LINESTRING (-77.03633567175983 38.987387049320...


In [12]:
crash_merged = crash_drop_gp.merge(dc_seg_gp, left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=False, how='left')

In [13]:
crash_merged = crash_merged.merge(dc_segments[['STREETSEGID', 'SHAPE_Length']], left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=False, how='left')

In [14]:
crash_merged.head().T

,0,1,2,3,4
CRASHID,2.84879e+08,2.84794e+08,2.84794e+08,2.84794e+08,2.84794e+08
STREETSEGID,2480,296,612,12926,628
geometry_x,POINT (-77.01991459279999 38.9569923951),POINT (-76.9763628826 38.8739935028),POINT (-77.0275123694 38.9287124837),POINT (-77.0398083253 38.8999813534),POINT (-77.0536723136 38.9034414121)
OBJECTID,305,10709,2374,2709,4455
geometry_y,LINESTRING (-77.01988963792071 38.956456767937...,LINESTRING (-76.97636288257532 38.873993502902...,LINESTRING (-77.02751236956541 38.928712483718...,LINESTRING (-77.03945044684772 38.900215595946...,LINESTRING (-77.05330540817235 38.903643942147...
SHAPE_Length,118.998,113.594,191.017,195.386,45.2589


In [79]:
import math

In [15]:
crash_merged.shape[0] - crash_merged.dropna().shape[0]

1731

In [16]:
crash_merged = crash_merged.drop(['geometry_x', 'geometry_y'], 1)
crash_merged.head().T

,0,1,2,3,4
CRASHID,2.848785e+08,2.847935e+08,2.847936e+08,2.847936e+08,2.847936e+08
STREETSEGID,2.480000e+03,2.960000e+02,6.120000e+02,1.292600e+04,6.280000e+02
OBJECTID,3.050000e+02,1.070900e+04,2.374000e+03,2.709000e+03,4.455000e+03
SHAPE_Length,1.189985e+02,1.135944e+02,1.910169e+02,1.953863e+02,4.525886e+01


In [20]:
crash_merged.shape

(122354, 4)

In [17]:
crash_statistic = crash_merged.merge(crash_gpd[['CRASHID', 'CRASHEVENTTYPES', 'CYCLISTSINVOLVED', 'FATALITIES', 'FIRSTHARMFULEVENTSPECIFICS', 'ISDRINKING', 'PEDESTRIANSINVOLVED', 'STREETLIGHTING']], left_on = 'CRASHID', right_on = 'CRASHID', right_index=False, how='left')

In [18]:
print(crash_statistic.shape)
crash_statistic.head().T

(122354, 11)


,0,1,2,3,4
CRASHID,2.84879e+08,2.84794e+08,2.84794e+08,2.84794e+08,2.84794e+08
STREETSEGID,2480,296,612,12926,628
OBJECTID,305,10709,2374,2709,4455
SHAPE_Length,118.998,113.594,191.017,195.386,45.2589
CRASHEVENTTYPES,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION","COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION","COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION","COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION","COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION"
CYCLISTSINVOLVED,None,None,None,None,None
FATALITIES,None,None,None,None,None
FIRSTHARMFULEVENTSPECIFICS,Motor Vehicle In Transport,Motor Vehicle In Transport,Motor Vehicle In Transport,Motor Vehicle In Transport,Motor Vehicle In Transport
ISDRINKING,No data provided,No data provided,No data provided,No data provided,No data provided
PEDESTRIANSINVOLVED,None,None,None,None,None


In [19]:
unique_crash_event_types = set()
for event in crash_statistic.CRASHEVENTTYPES:
    unique_crash_event_types.add(event)
print(len(unique_crash_event_types), unique_crash_event_types)

16 {'COLLISION_WITH_FIXED, NON_COLLISION, PENDING_INVESTIGATION', 'COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION', 'COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED', 'COLLISION_WITH_NON_FIXED, NON_COLLISION', 'NON_COLLISION', 'COLLISION_WITH_FIXED, NON_COLLISION', 'NON_COLLISION, PENDING_INVESTIGATION', 'COLLISION_WITH_FIXED', 'PENDING_INVESTIGATION', 'COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, NON_COLLISION, PENDING_INVESTIGATION', 'COLLISION_WITH_NON_FIXED', 'COLLISION_WITH_NON_FIXED, NON_COLLISION, PENDING_INVESTIGATION', 'COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION', 'COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, NON_COLLISION', 'No data provided', 'COLLISION_WITH_FIXED, PENDING_INVESTIGATION'}


In [20]:
unique_fhes = set()
for event in crash_statistic.FIRSTHARMFULEVENTSPECIFICS:
    unique_fhes.add(event)
print(len(unique_fhes), unique_fhes)

44 {'Tree(standing)', 'Concrete Traffic Barrier', 'Other Post, Pole Or Support', 'Mailbox', 'Jackknife', 'Guardrail End', 'Hit and Run', 'Other Non-fixed Object', 'Cargo/Equipment Loss Or Shift', 'Animal', 'Fire/Explosion', 'Other Fixed Object (Wall, Building, Tunnel, Etc.)', 'Commercial Vehicle', 'Bridge Rail', 'Pedestrian', 'Fell/Jumped From Motor Vehicle', 'Impact Attenuator/Crash Cushion', 'Struck By Falling, Shifting Cargo Or Anything Set In Motion By Motor Vehicle', 'Overturn/Rollover', 'Curb', 'Other Non-collision', 'Guardrail Face', 'Injury', 'Motor Vehicle In Transport', 'Utility Pole/light Support', 'Traffic Signal Support', 'Fatality', 'Other Property Damage', 'Bridge Overhead Structure', 'Culvert', 'Fence', 'Ditch', 'Bridge Pier Or Support', 'Parked Motor Vehicle', 'Traffic Sign Support', 'Unknown', 'Other Traffic Barrier', 'null', 'Railway Vehicle (train, engine)', 'D.C. Property', 'Cable Barrier', 'Pedalcycle', 'Embankment', 'Work Zone/maintenance Equipment'}


In [101]:
crash_statistic[crash_statistic.STREETSEGID==2480]

,CRASHID,STREETSEGID,OBJECTID,SHAPE_Length,CRASHEVENTTYPES,CYCLISTSINVOLVED,FATALITIES,FIRSTHARMFULEVENTSPECIFICS,ISDRINKING,PEDESTRIANSINVOLVED,STREETLIGHTING
0,284878528.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On
309,284795093.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,null,No data provided,None,Street Lights Off
506,284797373.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Parked Motor Vehicle,No data provided,None,Street Lights Off
7089,285594380.0,2480.0,305.0,118.998473,COLLISION_WITH_NON_FIXED,None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off
33926,289892181.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off
38848,290329548.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off
43166,291049567.0,2480.0,305.0,118.998473,"COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED...",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On
52116,292185114.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On
53413,292448738.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off
63133,294297732.0,2480.0,305.0,118.998473,PENDING_INVESTIGATION,None,None,Hit and Run,No data provided,None,Street Lights On


In [21]:
crash_count = crash_statistic.groupby('STREETSEGID').size().reset_index()

In [100]:
crash_count[crash_count.STREETSEGID==2480]

,STREETSEGID,crash_count
1981,2480.0,24


In [22]:
crash_count.columns = ['STREETSEGID', 'crash_count']
crash_count.head().T

,0,1,2,3,4
STREETSEGID,-11.0,1.0,2.0,3.0,4.0
crash_count,1137.0,7.0,14.0,12.0,15.0


In [23]:
crash_gpd[crash_gpd.STREETSEGID=='-11.0'].T

,85,252,691,733,801,826,843,886,893,991,...,118698,118840,119521,119783,119827,119978,121189,122171,122688,124025
ADDRESS1,7TH ST NW & K ST NW,I 395 & MAINE AVE SW,I 295 & MALCOLM X AVE SE,THEODORE ROOSEVELT BRIDGE,7TH ST NW & K ST NW,WASHINGTON CIRCLE,I 295 & MALCOLM X AVE SE,I 395 & MAINE AVE SW,WASHINGTON CIRCLE,I 395 & MAINE AVE SW,...,THOMAS CIRCLE NW,THOMAS CIRCLE NW,THOMAS CIRCLE NW,THOMAS CIRCLE NW,THOMAS CIRCLE NW,THOMAS CIRCLE NW,EASTERN AVE NE / KENILWORTH AVE NE,THOMAS CIRCLE NW,THOMAS CIRCLE NW,THOMAS CIRCLE NW
ADDRESS2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
ADDRESS_ID,900796,913126,913182,303204,900796,294277,913182,913126,294277,913126,...,310386,310386,310386,310386,310386,310386,904150,310386,310386,310386
CRASHEVENTTYPES,PENDING_INVESTIGATION,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION","COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED...","COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",PENDING_INVESTIGATION,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",PENDING_INVESTIGATION,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",COLLISION_WITH_NON_FIXED,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",...,COLLISION_WITH_NON_FIXED,COLLISION_WITH_NON_FIXED,COLLISION_WITH_NON_FIXED,COLLISION_WITH_FIXED,COLLISION_WITH_NON_FIXED,COLLISION_WITH_NON_FIXED,COLLISION_WITH_NON_FIXED,COLLISION_WITH_NON_FIXED,COLLISION_WITH_NON_FIXED,COLLISION_WITH_NON_FIXED
CRASHID,2.84948e+08,2.84797e+08,2.84796e+08,2.84875e+08,2.85027e+08,2.85025e+08,2.84948e+08,2.85026e+08,2.85026e+08,2.8495e+08,...,4.06939e+08,4.10058e+08,4.16803e+08,4.16406e+08,4.18086e+08,4.18163e+08,4.16462e+08,4.08507e+08,4.06634e+08,4.16242e+08
CRASHSCENEIMAGE,null,null,null,null,null,null,null,null,null,null,...,null,null,null,null,null,null,null,null,null,null
CRIMEID,2.84953e+08,2.84794e+08,2.84793e+08,2.84875e+08,2.85025e+08,2.85027e+08,2.84953e+08,2.85025e+08,2.85025e+08,2.84949e+08,...,4.06937e+08,4.10056e+08,4.16745e+08,4.16405e+08,4.18086e+08,4.18162e+08,4.16462e+08,4.08507e+08,4.06634e+08,4.16241e+08
CYCLISTSINVOLVED,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
DCLOCATIONBEARING,null,null,null,null,null,null,null,null,null,null,...,null,null,null,null,null,null,null,null,null,null
DCLOCATIONBRIDGE,null,null,null,null,null,null,null,null,null,null,...,null,null,null,null,null,null,null,null,null,null


In [24]:
crash_statistic[pd.isnull(crash_statistic.SHAPE_Length)==0].shape

(120623, 11)

In [25]:
crash_statistic.shape[0] - crash_statistic[pd.isnull(crash_statistic.SHAPE_Length)==0].shape[0]

1731

In [26]:
crash_statistic[pd.isnull(crash_statistic.SHAPE_Length)]

,CRASHID,STREETSEGID,OBJECTID,SHAPE_Length,CRASHEVENTTYPES,CYCLISTSINVOLVED,FATALITIES,FIRSTHARMFULEVENTSPECIFICS,ISDRINKING,PEDESTRIANSINVOLVED,STREETLIGHTING
81,284948143.0,-11.0,NaN,NaN,PENDING_INVESTIGATION,None,None,null,No data provided,None,Street Lights On
247,284796750.0,-11.0,NaN,NaN,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On
307,284795042.0,2731.0,NaN,NaN,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Hit and Run,No data provided,None,Street Lights Off
338,284797125.0,298.0,NaN,NaN,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off
397,284873484.0,15424.0,NaN,NaN,COLLISION_WITH_NON_FIXED,None,None,Parked Motor Vehicle,No data provided,None,Street Lights On
683,284796027.0,-11.0,NaN,NaN,"COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED...",None,None,"Other Fixed Object (Wall, Building, Tunnel, Etc.)",No data provided,None,Street Lights On
716,284874839.0,1879.0,NaN,NaN,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off
725,284874961.0,-11.0,NaN,NaN,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off
790,285026505.0,-11.0,NaN,NaN,PENDING_INVESTIGATION,None,None,null,No data provided,None,Other
815,285024932.0,-11.0,NaN,NaN,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off


In [27]:
crash_stats = crash_statistic[pd.isnull(crash_statistic.SHAPE_Length)==0]

In [99]:
crash_stats[crash_stats.STREETSEGID==2480]

,CRASHID,STREETSEGID,OBJECTID,SHAPE_Length,CRASHEVENTTYPES,CYCLISTSINVOLVED,FATALITIES,FIRSTHARMFULEVENTSPECIFICS,ISDRINKING,PEDESTRIANSINVOLVED,STREETLIGHTING,crash_count
0,284878528.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On,24
306,284795093.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,null,No data provided,None,Street Lights Off,24
501,284797373.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Parked Motor Vehicle,No data provided,None,Street Lights Off,24
7001,285594380.0,2480.0,305.0,118.998473,COLLISION_WITH_NON_FIXED,None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off,24
33596,289892181.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off,24
38447,290329548.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off,24
42712,291049567.0,2480.0,305.0,118.998473,"COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED...",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On,24
51518,292185114.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On,24
52784,292448738.0,2480.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off,24
62352,294297732.0,2480.0,305.0,118.998473,PENDING_INVESTIGATION,None,None,Hit and Run,No data provided,None,Street Lights On,24


In [28]:
crash_count_drop = crash_stats.groupby('STREETSEGID').size().reset_index()
crash_count_drop.columns = ['STREETSEGID', 'crash_count']
crash_count_drop.head().T

,0,1,2,3,4
STREETSEGID,1.0,2.0,3.0,4.0,5.0
crash_count,7.0,14.0,12.0,15.0,66.0


In [102]:
crash_count_drop[crash_count_drop.STREETSEGID==2480]

,STREETSEGID,crash_count
1968,2480.0,24


In [103]:
crash_stats = crash_count_drop.merge(crash_stats, left_on='STREETSEGID', right_on='STREETSEGID', right_index=False, how='left')

In [104]:
crash_stats[crash_stats.SHAPE_Length==0]

,STREETSEGID,crash_count_x,CRASHID,OBJECTID,SHAPE_Length,CRASHEVENTTYPES,CYCLISTSINVOLVED,FATALITIES,FIRSTHARMFULEVENTSPECIFICS,ISDRINKING,PEDESTRIANSINVOLVED,STREETLIGHTING,crash_count_y


In [105]:
crash_stats.head().T

,0,1,2,3,4
STREETSEGID,1,1,1,1,1
crash_count_x,7,7,7,7,7
CRASHID,2.90331e+08,2.90253e+08,2.91934e+08,2.97409e+08,2.98315e+08
OBJECTID,10919,10919,10919,10919,10919
SHAPE_Length,180.261,180.261,180.261,180.261,180.261
CRASHEVENTTYPES,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",COLLISION_WITH_FIXED,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",PENDING_INVESTIGATION,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION"
CYCLISTSINVOLVED,None,None,None,None,None
FATALITIES,None,None,None,None,None
FIRSTHARMFULEVENTSPECIFICS,Parked Motor Vehicle,"Other Fixed Object (Wall, Building, Tunnel, Etc.)",Parked Motor Vehicle,Hit and Run,Parked Motor Vehicle
ISDRINKING,No data provided,No data provided,No data provided,No data provided,No data provided


In [106]:
crash_stats.shape

(120623, 13)

In [107]:
crash_stats[crash_stats.STREETSEGID==2480]

,STREETSEGID,crash_count_x,CRASHID,OBJECTID,SHAPE_Length,CRASHEVENTTYPES,CYCLISTSINVOLVED,FATALITIES,FIRSTHARMFULEVENTSPECIFICS,ISDRINKING,PEDESTRIANSINVOLVED,STREETLIGHTING,crash_count_y
32808,2480.0,24,284878528.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On,24
32809,2480.0,24,284795093.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,null,No data provided,None,Street Lights Off,24
32810,2480.0,24,284797373.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Parked Motor Vehicle,No data provided,None,Street Lights Off,24
32811,2480.0,24,285594380.0,305.0,118.998473,COLLISION_WITH_NON_FIXED,None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off,24
32812,2480.0,24,289892181.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off,24
32813,2480.0,24,290329548.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off,24
32814,2480.0,24,291049567.0,305.0,118.998473,"COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED...",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On,24
32815,2480.0,24,292185114.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights On,24
32816,2480.0,24,292448738.0,305.0,118.998473,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",None,None,Motor Vehicle In Transport,No data provided,None,Street Lights Off,24
32817,2480.0,24,294297732.0,305.0,118.998473,PENDING_INVESTIGATION,None,None,Hit and Run,No data provided,None,Street Lights On,24


In [33]:
crash_typemerge = crash_stats.groupby(['STREETSEGID', 'CRASHEVENTTYPES']).size().reset_index()
crash_typemerge.columns = ['STREETSEGID', 'CRASHEVENTTYPES', 'count']
print(crash_typemerge.shape)
crash_typemerge.head().T

(28238, 3)


,0,1,2,3,4
STREETSEGID,1,1,1,1,2
CRASHEVENTTYPES,COLLISION_WITH_FIXED,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION",NON_COLLISION,PENDING_INVESTIGATION,"COLLISION_WITH_FIXED, NON_COLLISION, PENDING_I..."
count,1,4,1,1,1


In [95]:
crash_details = crash_stats[['STREETSEGID', 'SHAPE_Length', 'crash_count']]

In [128]:
print(crash_details.shape)
crash_details.head().T

(9695, 3)


,0,1,2,3,4
STREETSEGID,2480.000000,296.000000,612.000000,12926.000000,628.000000
SHAPE_Length,118.998473,113.594389,191.016915,195.386261,45.258864
crash_count,24.000000,4.000000,42.000000,48.000000,85.000000


In [129]:
crash_details[crash_details.STREETSEGID==2480]

,STREETSEGID,SHAPE_Length,crash_count
0,2480.0,118.998473,24


In [130]:
for event in unique_crash_event_types:
    event_data = crash_typemerge[crash_typemerge.CRASHEVENTTYPES==event]
    event_count = event + '_count'
    event_data_count = event_data[['STREETSEGID', 'count']]
    event_data_count.columns = ['STREETSEGID', event_count]
    crash_details = crash_details.merge(event_data_count, left_on = 'STREETSEGID', right_on = 'STREETSEGID', right_index=False, how='left')

In [131]:
crash_details = crash_details.fillna(0)
print(crash_details.shape)
crash_details.head().T

(9695, 19)


,0,1,2,3,4
STREETSEGID,2480.000000,296.000000,612.000000,12926.000000,628.000000
SHAPE_Length,118.998473,113.594389,191.016915,195.386261,45.258864
crash_count,24.000000,4.000000,42.000000,48.000000,85.000000
"COLLISION_WITH_FIXED, NON_COLLISION, PENDING_INVESTIGATION_count",0.000000,0.000000,0.000000,0.000000,0.000000
"COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION_count",1.000000,0.000000,0.000000,0.000000,0.000000
"COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED_count",0.000000,0.000000,0.000000,0.000000,0.000000
"COLLISION_WITH_NON_FIXED, NON_COLLISION_count",0.000000,0.000000,1.000000,0.000000,0.000000
NON_COLLISION_count,0.000000,0.000000,0.000000,0.000000,0.000000
"COLLISION_WITH_FIXED, NON_COLLISION_count",0.000000,0.000000,0.000000,0.000000,0.000000
"NON_COLLISION, PENDING_INVESTIGATION_count",0.000000,0.000000,1.000000,0.000000,0.000000


In [132]:
crash_details.shape

(9695, 19)

In [133]:
crash_details.columns

Index(['STREETSEGID', 'SHAPE_Length', 'crash_count',
       'COLLISION_WITH_FIXED, NON_COLLISION, PENDING_INVESTIGATION_count',
       'COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION_count',
       'COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED_count',
       'COLLISION_WITH_NON_FIXED, NON_COLLISION_count', 'NON_COLLISION_count',
       'COLLISION_WITH_FIXED, NON_COLLISION_count',
       'NON_COLLISION, PENDING_INVESTIGATION_count',
       'COLLISION_WITH_FIXED_count', 'PENDING_INVESTIGATION_count',
       'COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, NON_COLLISION, PENDING_INVESTIGATION_count',
       'COLLISION_WITH_NON_FIXED_count',
       'COLLISION_WITH_NON_FIXED, NON_COLLISION, PENDING_INVESTIGATION_count',
       'COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION_count',
       'COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, NON_COLLISION_count',
       'No data provided_count',
       'COLLISION_WITH_FIXED, PENDING_INVESTIGATION_count'],
      dtype='o

In [134]:
def normalize(col_name, df):
    col_name_norm = col_name + '_norm'
    df[col_name_norm] = df[col_name]+1
    df[col_name_norm] = np.log2(df[col_name_norm])
    df[col_name_norm] = df[col_name_norm]/df[col_name_norm].max()

In [135]:
for column in crash_details.columns:
    if('_count' in column):
        col_len = column + '/len'
        crash_details[col_len] = crash_details[column]/crash_details.SHAPE_Length
        normalize(column, crash_details)
        normalize(col_len, crash_details)

In [136]:
crash_details = crash_details.drop_duplicates()
print(crash_details.shape)
crash_details.head().T

(9695, 70)


,0,1,2,3,4
STREETSEGID,2480.000000,296.000000,612.000000,12926.000000,628.000000
SHAPE_Length,118.998473,113.594389,191.016915,195.386261,45.258864
crash_count,24.000000,4.000000,42.000000,48.000000,85.000000
"COLLISION_WITH_FIXED, NON_COLLISION, PENDING_INVESTIGATION_count",0.000000,0.000000,0.000000,0.000000,0.000000
"COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION_count",1.000000,0.000000,0.000000,0.000000,0.000000
"COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED_count",0.000000,0.000000,0.000000,0.000000,0.000000
"COLLISION_WITH_NON_FIXED, NON_COLLISION_count",0.000000,0.000000,1.000000,0.000000,0.000000
NON_COLLISION_count,0.000000,0.000000,0.000000,0.000000,0.000000
"COLLISION_WITH_FIXED, NON_COLLISION_count",0.000000,0.000000,0.000000,0.000000,0.000000
"NON_COLLISION, PENDING_INVESTIGATION_count",0.000000,0.000000,1.000000,0.000000,0.000000


In [142]:
crash_details.to_csv('../DC Crash Details.csv')

In [138]:
crash_details[crash_details['COLLISION_WITH_FIXED, NON_COLLISION, PENDING_INVESTIGATION_count']==0].shape

(9387, 70)

In [139]:
crash_details.shape

(9695, 70)

In [141]:
crash_details[crash_details.STREETSEGID==2480]

,STREETSEGID,SHAPE_Length,crash_count,"COLLISION_WITH_FIXED, NON_COLLISION, PENDING_INVESTIGATION_count","COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION_count","COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED_count","COLLISION_WITH_NON_FIXED, NON_COLLISION_count",NON_COLLISION_count,"COLLISION_WITH_FIXED, NON_COLLISION_count","NON_COLLISION, PENDING_INVESTIGATION_count",...,"COLLISION_WITH_NON_FIXED, PENDING_INVESTIGATION_count/len_norm","COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, NON_COLLISION_count/len","COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, NON_COLLISION_count_norm","COLLISION_WITH_FIXED, COLLISION_WITH_NON_FIXED, NON_COLLISION_count/len_norm",No data provided_count/len,No data provided_count_norm,No data provided_count/len_norm,"COLLISION_WITH_FIXED, PENDING_INVESTIGATION_count/len","COLLISION_WITH_FIXED, PENDING_INVESTIGATION_count_norm","COLLISION_WITH_FIXED, PENDING_INVESTIGATION_count/len_norm"
0,2480.0,118.998473,24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.063863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
unique_rows = set()
for segid in crash_details.STREETSEGID:
    unique_rows.add(segid)
print(len(unique_rows), unique_rows)

9695 {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 22.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 39.0, 40.0, 41.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 101.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 112.0, 113.0, 114.0, 115.0, 116.0, 118.0, 119.0, 120.0, 121.0, 123.0, 124.0, 125.0, 126.0, 127.0, 129.0, 130.0, 132.0, 133.0, 134.0, 135.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 148.0, 150.0, 151.0, 153.0, 155.0, 156.0, 158.0, 159.0, 160.0, 161.0, 162.0, 163.0, 164.0, 165.0, 166.0, 168.0, 170.0, 171.0, 172.0, 173.0, 174.0, 175.0, 176.0, 177.0, 178.0, 179.0, 180.0, 181

In [110]:
crash_details.drop_duplicates()[crash_details.STREETSEGID==2480]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,STREETSEGID,SHAPE_Length,crash_count
0,2480.0,118.998473,24


In [28]:
dc_city_req_details = pd.DataFrame(pd.read_csv('../DC_City_Request_Details_filtered.csv'))

In [29]:
dc_city_req_details.shape

(13522, 241)

In [30]:
dc_city_req_details.drop_duplicates().shape

(13522, 241)

In [32]:
dc_city_req_details = pd.DataFrame(pd.read_csv('../DC_City_Request_Details_complete.csv'))
dc_city_req_details.shape

(13522, 245)

In [33]:
dc_city_req_details.drop_duplicates().shape

(13522, 245)

In [34]:
dc_city_req_details = dc_city_req_details.drop('Unnamed: 0', axis=1)

In [ ]:
dc_city_req_details['city_req_count/len']

In [35]:
dc_city_req_details.head().T

,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
city_req_seg_count,3,2,80,6,2
STREET PAVING SCHEDU_count,0,0,0,0,0
LIGHT-TUNNEL/UNDPASS_count,0,0,0,0,0
RESIDENTIAL PERMIT -_count,0,0,0,0,0
PUBLIC SPACE - OBSTR_count,0,0,0,0,0
SIGN NEW_count,0,0,0,0,0


In [6]:
dc_vision0_details=pd.read_csv('../DC_Vision_Zero_Details.csv')
dc_vision0_details.shape

(15071, 76)

In [7]:
dc_vision0_details.drop_duplicates().shape

(15071, 76)

In [10]:
dc_vision0_details = dc_vision0_details.drop('Unnamed: 0', axis=1)

In [11]:
dc_vision0_details.head().T

,0,1,2,3,4
OBJECTID,1.000000,2.000000,3.000000,4.000000,5.000000
STREETSEGID,876.000000,13161.000000,8240.000000,10395.000000,2216.000000
SHAPE_Length,192.925131,131.935677,117.887011,108.331118,119.190312
vision0_pts_seg,2.000000,1.000000,0.000000,0.000000,0.000000
vision0_pts/len,0.010367,0.007579,0.000000,0.000000,0.000000
vision0_pts_seg_norm,0.350379,0.221065,0.000000,0.000000,0.000000
vision0_pts/len_norm,0.030195,0.022107,0.000000,0.000000,0.000000
Red light running_count,0.000000,0.000000,0.000000,0.000000,0.000000
Accessibility Issue_count,0.000000,0.000000,0.000000,0.000000,0.000000
Failure to stop for pedestrians_count,0.000000,0.000000,0.000000,0.000000,0.000000


In [1]:
def gradient_color(percent):
    min_color = np.array([255,255,255])
    max_color = np.array([178,34,34])
    return '#%02x%02x%02x' % tuple([int(k) for k in min_color+(max_color-min_color)*percent])

def write_var(col_name, var_name, df,f):
    df['color']=df[col_name].apply(gradient_color)
    f.write('var %s = %s;\n' % (var_name, df.to_json()))

with open('../visualization/Volumes of City Requests with Parking Meter Req.js','w+') as f:
    write_var('city_req_seg_count_norm', 'city_req_count', dc_city_req_details,f)
    write_var('city_req_count/len_norm', 'city_req_count_norm', dc_city_req_details,f)
    #write_var('Parking Meter_count_norm', 'Parking_Meter_count', dc_city_req_details,f)
    #write_var('Parking_Meter_count/len_norm', 'Parking_Meter_count_norm', dc_city_req_details,f)

NameError: name 'dc_city_req_details' is not defined